# Lights, Camera, ReAction ✨🎥😆🎬


**ADD HOW INFO IS STORED IN META DATA**

This notebook builds a full **sitcom script generation pipeline** powered by **ReAct-based agents** — a powerful framework that combines **reasoning** (thinking through a problem) with **acting** (taking structured steps).

We start by **generating a sitcom concept** from creative keywords, then **outlining** the pilot episode scene-by-scene.  
**Scene 1** is generated directly from the outline to establish the world and tone.  
After that, each new scene is **scripted, reviewed, and improved** using specialized **ReAct agents** — a **Character Agent**, **Comedy Agent**, and **Environment Agent** — that simulate a real sitcom writers' room.

---

 **Benefits of using ReAct agents**:

- **More structured and transparent thinking:** Agents reason step-by-step before making edits.
- **Dynamic adaptation:** Agents flexibly plan the next creative moves based on scene history.
- **Better long-term coherence:** Scenes evolve logically, with tracked character growth, running jokes, emotional arcs, and worldbuilding.

---

After generation, each scene is **summarized and stored in a vector database**, enabling fast retrieval of scene metadata for future story planning.  
By combining **structured agent workflows** and **retrieval-augmented memory**, we bring sitcom worlds to life — one coherent, character-driven scene at a time.


In [ ]:
!pip install datasets
!pip install sentence-transformers faiss-cpu

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 31.3/31.3 MB 17.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 68.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 52.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 37.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 12.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 6.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 59.5 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstallin

## Mounting Drive and Appending System

In [ ]:
# Import necessary modules
from google.colab import drive
import sys

# Mount Google Drive to access files stored in your Google Drive
drive.mount('/content/drive')

# NOTE: Update the paths below to match the location of your project files in Google Drive.
# Replace with your own directory if different.

# Add the main 'utils' directory to Python's module search path
# This allows you to import custom utility modules from this folder
sys.path.append("/content/drive/MyDrive/Spring 2025/Gen AI with LLM/Project/utils")

# Add the 'agents' subdirectory to the module search path
# Useful if you have agent-specific Python scripts you want to import
sys.path.append("/content/drive/MyDrive/Spring 2025/Gen AI with LLM/Project/utils/agents")

Mounted at /content/drive


In [ ]:
# TO DELETE

!ls "/content/drive/MyDrive/Spring 2025/Gen AI with LLM/Project/utils"


agents		       __pycache__	  script_review.py  vector_db_utils.py
outline_generation.py  screen_writing.py  text_utils.py


In [ ]:
'''
TO DELETE
'''

import importlib
import agent_evaluation

importlib.reload(agent_evaluation)


<module 'agent_evaluation' from '/content/drive/MyDrive/Spring 2025/Gen AI with LLM/Project/utils/agents/agent_evaluation.py'>

## Importing Libraries and Packages

In [ ]:
# Import sitcom agents responsible for generating specific scene elements
from agents.character_agent import CharacterAgent
from agents.comedy_agent import ComedicAgent
from agents.environment_agent import EnvironmentAgent
from agents.scene_planner_agent import ScenePlannerAgent

# Import evaluation functions to assess the quality of generated scenes
from agent_evaluation import (
    evaluate_character_agent_scene,
    evaluate_comedic_agent_scene,
    evaluate_environment_agent_scene
)

# Import text utility functions for formatting and extracting key components
from text_utils import (
    display_markdown_output,
    extract_scene,
    extract_title,
)

# Import functions for generating sitcom concepts and outlines
from outline_generation import (
    generate_sitcom_pitch,
    generate_pilot_episode_outline
)

# Import functions to validate the structure and consistency of episodes
from script_review import (
    validate_episode_outline,
)

# Import scene writing functions
from screen_writing import (
    generate_scene_1_script,
    generate_scene
)

# Import vector database utilities for storing and retrieving scene information
from vector_db_utils import (
    summarize_scene,
    add_scene_to_vector_db,
    store_scene_in_vector_db
)

In [ ]:
# Core Python Libraries
import os
import re

# Math and Data Manipulation
import numpy as np

# Display Utilities
from IPython.display import Markdown, display

# OpenAI API
import openai
from openai import OpenAI

# Google Colab Specific
from google.colab import userdata

# Sentence Embeddings and Vector Search
from sentence_transformers import SentenceTransformer
import faiss

### Package Versions

In [ ]:
import pkg_resources

print("numpy:", np.__version__)
print("openai:", openai.__version__)
print("sentence-transformers:", pkg_resources.get_distribution("sentence-transformers").version)
print("faiss:", faiss.__version__ if hasattr(faiss, '__version__') else "version attribute not available")

numpy: 2.0.2
openai: 1.77.0
sentence-transformers: 4.1.0
faiss: 1.11.0



### 🔑 Initializing the OpenAI API Client

In [ ]:
api_key = userdata.get('OPENAI_API_KEY')  # Securely fetch API key from Colab's stored credentials
client = OpenAI(api_key=api_key)          # Initialize the OpenAI client with the retrieved key

### Vector Database

This block initializes the sentence embedding model and sets up a FAISS index to store vector representations of scenes. The `all-MiniLM-L6-v2` model encodes scenes into fixed-size embeddings, and the FAISS index enables fast vector operations. The `scene_metadata` list stores contextual information linked to each scene embedding.


In [ ]:
# Load embedding model
embedding_model = SentenceTransformer("all-MiniLM-L6-v2")

# Vector store initialization
dimension = embedding_model.get_sentence_embedding_dimension()
index = faiss.IndexFlatL2(dimension)

# To track metadata
vector_metadata = []

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.5k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

### 🎬 Generating the Sitcom Pitch

This step generates a sitcom concept using the `generate_sitcom_pitch` function. The function takes in a dictionary of categorized keywords (e.g., setting, characters, themes, tone/genre) to guide the generation process.

- The language model plays the role of a **professional screenwriter**.
- It produces a 1-paragraph pitch that includes a **title** and a **detailed premise**, highlighting the setting, main characters, their dynamics, and the overall tone of the show.
- Keywords help ground the pitch in user-defined ideas, ensuring thematic and narrative consistency.

In [ ]:
# Define key elements for sitcom generation: setting, characters, themes, and tone
keywords = {
    "setting": ["urban locksmith shop", "New York City"],
    "characters": ["ex-con protagonist", "estranged daughter", "quirky parole officer"],
    "themes": ["second chances", "odd couple dynamic"],
    "tone_genre": ["buddy comedy", "heartfelt absurdism"]
}

In [ ]:
# Generate sitcom pitch using the language model
sitcom_pitch = generate_sitcom_pitch(client, keywords)

# Display the generated outline in formatted Markdown
display(Markdown(f"### Sitcom Concept:\n\n{sitcom_pitch}"))


### Sitcom Concept:

Title: "Locked In"

"Locked In" is a heartfelt yet absurdly comedic sitcom set in the bustling heart of New York City, revolving around the life of Joe, a recently paroled ex-con who's trying to turn over a new leaf by running an urban locksmith shop. Joe's life takes an unexpected turn when his estranged daughter, Casey, a free-spirited artist with her own set of lock-picking skills, moves in with him, sparking an odd couple dynamic as they navigate their rekindled relationship. Adding to the hilarity is Joe's quirky parole officer, Doug, who becomes a frequent visitor to the shop under the guise of random check-ins but is secretly fascinated by the art of locksmithing. With themes of second chances, familial bonds, and the daily eccentricities of city living, "Locked In" offers a unique blend of buddy comedy and heartfelt storytelling that explores the key to unlocking a brighter future.

#### Extracting Title

Below, we extract the sitcom title from the generated pitch. This title will be reused by the script generation functions (both Baseline and ReAct versions).


In [ ]:
# Extract title
sitcom_title = extract_title(sitcom_pitch)
print(sitcom_title)

Locked In


### 📝 Generating a Pilot Episode Outline

The next part is to generate an episode outline for the pilot of the show, which is done using the `generate_pilot_episode_outline` function. This function takes in the sitcom pitch and produces a structured scene-by-scene breakdown of the pilot episode, simulating the voice of a **professional sitcom writer** preparing a network pitch.

- Begins with a concise **Episode Concept** (1–2 sentences).
- Breaks the episode into approximately 20 **numbered scenes**, each with a **title in quotation marks** and a brief description.
- Each scene summary includes what happens, who is involved, and the intended tone (e.g., funny, awkward, heartfelt).

This outline provides the narrative foundation for generating full scene scripts in the next stage.


In [ ]:
# Generate episode outline from sitcom pitch using the language model
episode_outline = generate_pilot_episode_outline(client, sitcom_pitch)

# Display the generated outline in formatted Markdown
display(Markdown(f"### Sitcom Outline:\n\n{episode_outline}"))

### Sitcom Outline:

Episode Concept: Joe's first day back to his locksmith shop is interrupted by the unexpected arrival of his estranged daughter Casey, setting off a chain of humorous and heartfelt events as they navigate their reconnection and Joe's parole officer Doug's obsession with locksmithing.

Scene 1: "Fresh Start" 
Joe, an ex-con, opens his locksmith shop for the first time since his parole, setting a hopeful yet nervous tone.

Scene 2: "Unexpected Visitor"
Casey, Joe's estranged daughter, arrives at the shop, creating an awkward and surprising encounter.

Scene 3: "Art of Picking Locks"
Casey impresses Joe with her own lock-picking skills, generating some humor and intrigue.

Scene 4: "The Proposal"
Casey suggests moving in with Joe, creating a moment of surprise and tension.

Scene 5: "Welcome, Officer Doug"
Doug, Joe's quirky parole officer, makes his first unexpected visit to the shop, adding a comedic twist.

Scene 6: "A Shared Obsession"
Doug reveals his fascination with locksmithing to Joe, resulting in a humorous exchange.

Scene 7: "Casey's New Home"
Joe reluctantly agrees to let Casey move in, setting a bittersweet tone.

Scene 8: "House Rules"
Joe lays down some ground rules for Casey, creating a funny and awkward dynamic.

Scene 9: "Officer Doug's Second Visit"
Doug makes another unannounced visit, this time to check on Casey, resulting in some laughs.

Scene 10: "Late Night Locksmithing"
Doug convinces Joe to teach him locksmithing after hours, generating a comedic bonding moment.

Scene 11: "Father-Daughter Bonding"
Casey and Joe share a heartfelt conversation about their past and future.

Scene 12: "First Customer"
Joe helps a frantic customer with a broken lock, adding a touch of humor and normalcy.

Scene 13: "Casey's Contribution"
Casey uses her artistic skills to improve the shop's appearance, setting a hopeful and cheerful tone.

Scene 14: "The Officer's First Lock"
Doug picks his first lock under Joe's guidance, creating a triumphant and humorous scene.

Scene 15: "Breaking the Rules"
Casey breaks one of Joe's house rules, leading to a funny and tense moment.

Scene 16: "Officer Doug's Surprise Inspection"
Doug performs a surprise inspection at the shop, generating laughs and suspense.

Scene 17: "Father-Daughter Showdown"
Joe confronts Casey about breaking his rules, leading to a heartfelt and serious conversation.

Scene 18: "Locksmith Lesson"
Joe uses locksmithing as a metaphor to teach Casey a life lesson, creating a touching father-daughter moment.

Scene 19: "Doug's Locksmithing Progress"
Doug shows off his improved locksmithing skills, generating laughs and admiration.

Scene 20: "Locking In"
Joe, Casey, and Doug share a heartfelt moment of camaraderie in the shop, setting a hopeful and warm tone as they look forward to their shared future.

#### Exporting Outline and Title Name

Below, we export the sitcom title and episode outline. This ensures that both the Baseline and ReAct models use the same input for consistency in script generation.

In [ ]:
# Define file paths for saving the sitcom title and episode outline
title_path = "/content/drive/MyDrive/Spring 2025/Gen AI with LLM/Project/episode data/sitcom_title.txt"
outline_path = "/content/drive/MyDrive/Spring 2025/Gen AI with LLM/Project/episode data/episode_outline.txt"

# Write the sitcom title to a text file
with open(title_path, "w") as f:
    f.write(sitcom_title)

# Write the episode outline to a text file
with open(outline_path, "w") as f:
    f.write(episode_outline)


### ✅ Episode Validation

To ensure that the sitcom pilot outline is coherent and aligns with the original pitch, we use the `validate_episode_outline` function. This function simulates the voice of a **veteran sitcom writer** who evaluates whether the episode outline makes narrative sense and fits the tone and premise.

- Checks for overall **coherence**, including consistency of tone, logical character progression, and structural viability.
- Returns a clear **"Yes" or "No"** on coherence, along with 2–4 bullet points of reasoning.
- If the episode is not coherent, it also provides concrete suggestions for improvement.

This validation step helps maintain narrative quality before moving into full scene scripting.


**TODO**

- Make sure it can re-generate the script
- Delte timedelta

In [ ]:
# Validate the episode outline for coherence and consistency using the language model
validation = validate_episode_outline(client, sitcom_pitch, episode_outline)

# Display the validation result in formatted Markdown
display(Markdown(f"### Coherence Evaluation:\n\n{validation}"))

### Coherence Evaluation:

Coherence: Yes

Reasoning:
- The episode outline is consistent with the sitcom concept, as it introduces the main characters, their relationships, and the setting in a way that combines comedy with heartfelt moments.
- The plot progression is logical, with Casey's arrival at the shop serving as the inciting incident that sets off a series of events related to her reconnection with Joe and Doug's interest in locksmithing.
- The tone of the episode is consistent with the sitcom's premise, blending humor with more serious moments that explore themes of second chances and familial bonds.
- The episode outline sets up potential for future storylines, such as Casey's integration into Joe's life and Doug's continued interest in locksmithing.

## Scene 1 Generation

In [ ]:
# Extract the description for Scene 1 from the episode outline
scene_1_desc = extract_scene(episode_outline, 1)
print(scene_1_desc)

Scene 1: "Fresh Start" 
Joe, an ex-con, opens his locksmith shop for the first time since his parole, setting a hopeful yet nervous tone.

In [ ]:
# Generate the full script for Scene 1 based on the structured scene plan
scene_1_script = generate_scene_1_script(
    client=client,
    sitcom_title=sitcom_title,
    scene_description=scene_1_desc,
    scene_index=0,
    rag_context=None,
)

print(scene_1_script)

# Scene 1

INT. JOE'S LOCKSMITH SHOP - DAY

[The shop is cluttered with keys, locks, and locksmith equipment. JOE, late 30s, a tough guy with a soft side, is arranging a display of padlocks.]

JOE
(to himself)
Okay, Joe, fresh start. Just gotta remember, this is a legitimate business, not a...well, y'know. 

[JOE looks at a photo of himself in prison attire, shakes his head, and turns the photo face down.]

[LAUGH TRACK]

JOE 
(continuing)
No more picking locks. Only making them.

[SFX: DOORBELL RINGS as a CUSTOMER, a nervous woman in her 50s, enters.]

CUSTOMER
Oh, thank heavens you're open! I've locked myself out of my house, and my cat's still inside!

JOE
(smiling)
Don't worry, ma'am. I'll get your cat out in no time.

[JOE starts gathering his tools.]

CUSTOMER
Oh, thank you. You're a lifesaver.

JOE
(murmuring to himself)
And a locksmith...

[LAUGH TRACK]

[The CUSTOMER looks around the shop, her eyes landing on the turned-down photo.]

CUSTOMER
(curious)
Who's that in the photo?

### 🗂️ Adding Metadata to the Vector Database

To extract structured information from each generated sitcom scene, we use th `summarize_scene` function. This function simulates the head writer of the show and analyzes the scene script to produce a concise summary along with key metadata.

- **Summary**: A 100–150 word overview of the scene’s key actions, character dynamics, and notable moments.
- **Characters**: A list of characters who appear or speak in the scene.
- **Location**: The main setting of the scene, if clearly identifiable (e.g., “locksmith shop,” “apartment hallway”).
- **Recurring Joke**: Any running gag or callback featured in the scene.
- **Emotional Tone**: A 1–2 word description of the scene’s mood (e.g., “chaotic,” “hopeful,” “awkward”).

The metadata is returned as a dictionary and appended to the `scene_metadata` list using the `add_scene_to_vector_db` function. This step is crucial: instead of supplying the full script to the next generation phase, we pass a concise summary and key elements to guide smoother, context-aware scene generation.


In [ ]:
# Store the generated Scene 1 script in the vector database for future retrieval and context
store_scene_in_vector_db(
    client=client,
    sitcom_title=sitcom_title,
    scene_script=scene_1_script,
    embedding_model=embedding_model,
    index=index,
    vector_metadata=vector_metadata
)

Total scenes stored in vector DB: 1 


Scene 1
Summary: The scene takes place in the morning in Robbie's apartment, which is situated above a locksmith shop. Robbie, a locksmith in his mid-thirties, wakes up to his alarm clock ringing. He initially misreads the time due to his poor eyesight, but after finding his glasses, he realizes he is late for a locksmith convention. In his rush to get ready, he trips over his tools and realizes he's wearing his shirt backwards. Before leaving, he grabs his locksmith ID badge, reminding himself that he needs it to enter the convention. The scene ends with a humorous note stuck to the now silent alarm clock, reminding Robbie to buy new glasses.
Characters: ['Robbie']
Location: Robbie's apartment above the locksmith shop
Recurring Joke: Robbie's poor eyesight and his clumsy, disorganized nature are recurring sources of humor in this scene.
Emotional Tone: Comedic


## 🎬 Scene 2 Generation

The next part is to generate Scene 2. This is one of the most important scenes in the script, as it’s where the ReAct (Reasoning + Acting) system comes into play.

Metadata from Scene 1 is retrieved and interpreted by the `CharacterAgent`, `ComedyAgent`, and `EnvironmentAgent`. Each agent reasons over the metadata independently and provides creative suggestions based on its area of expertise—such as character consistency, comedic dynamics, or environmental context.

These agent responses are then passed to the `ScenePlannerAgent`, which integrates their inputs and plans the next scene accordingly. This collaborative reasoning-and-acting process ensures that each scene builds logically and creatively on what came before.

The advantage of using ReAct-style agents over purely role-based agents is that each agent not only plays a role but also actively reasons and adapts based on prior context—making the generation more dynamic, context-aware, and narratively coherent.


In [ ]:
# Extract the description for Scene 2 from the episode outline
scene_2_desc = extract_scene(episode_outline, 2)
print(scene_2_desc)

Scene 2: "First Client of the Day" 
Robbie helps a forgetful elderly woman who has lost her keys, showcasing his genuine care and charm.


### 🎭 Character Agent

The first step is to analyze the characters using the `CharacterAgent`. This agent is responsible for tracking which characters appear in each scene, evaluating their consistency with past behavior, and recommending meaningful interactions to advance the story.

The `CharacterAgent` operates using a **ReAct workflow**, where each phase reflects a specific role in a sitcom writers’ room:

- **Think**: Identifies which characters are present in the current scene and determines the relevant prior scenes to draw context from — like a **Writers' Assistant** reviewing past episodes. This step also retrieves the relevant script metadata used to guide character evaluation.
- **Act**: Retrieves character histories by analyzing summaries from earlier scenes — like a **Script Coordinator** maintaining continuity.
- **Observe**: Checks whether characters behave consistently with their established traits and arcs — like a **Head Writer** reviewing for narrative logic.
- **Recommend**: Suggests specific character interactions that align with the scene’s tone or resolve inconsistencies — like a **Co-Executive Producer** steering character development.

This structure ensures character arcs are coherent, grounded, and creatively engaging throughout the scriptwriting process.


In [ ]:
import importlib
import agents.character_agent
importlib.reload(agents.character_agent)
from agents.character_agent import CharacterAgent


In [ ]:
# Initialize the character agent with the language model client, metadata store, and total number of scenes
character_agent = CharacterAgent(
    client=client,
    vector_metadata=vector_metadata,
    num_scenes=1
)

# Evaluate Scene 2 using the character agent and store the evaluation results
char_histories, char_is_consistent, char_explanation, char_recommendations, char_thoughts = evaluate_character_agent_scene(
    agent=character_agent,
    scene_description=scene_2_desc,
    scene_number=2
)

🔁 Agent will consider the last 1 scene(s) for context.

📚 Retrieving script metadata for scene(s): [1]
Scene 2 — Consistency: ✅ Consistent

Explanation:
2. Short Explanation Why: Robbie's actions in the planned scene are consistent with his established personality traits. His dedication to his profession is shown through his willingness to help the elderly woman. His charm and care are also demonstrated in this interaction. The elderly woman's forgetfulness aligns with Robbie's own forgetfulness, which could potentially lead to comedic situations in the future.

Interaction Improvement Recommendations:
1. Robbie, despite being rushed and disorganized, takes a moment to patiently explain the lock mechanism to the elderly woman. This not only showcases his professional dedication from Scene 1 but also his charm and caring nature as he goes above and beyond to help his client understand her lock better.
2. Robbie misplaces his own set of master keys in the process of helping the elderly w

### 🎤 Comedic Agent


The `ComedicAgent` is responsible for maintaining the tonal consistency of humor throughout the sitcom and making targeted punch-up recommendations to enhance comedic value.

This agent operates using a **ReAct workflow**, where each phase mimics a specific role in a sitcom writers’ room and draws on **prior scene metadata** for context:

- **Think**: Logs which scene is under review and determines which previous scenes will provide comedic context — like a **Writers' Assistant** flagging what material to reference.
- **Act**: Retrieves summaries and running jokes from recent scenes by referencing stored **vector metadata** — like a **Script Coordinator** tracking comedic beats.
- **Observe**: Evaluates whether the new scene's humor is consistent with previous tone and joke usage — like a **Head Writer** preserving the show's comedic style.
- **Recommend**: Suggests two realistic comedic improvements grounded in the scene’s context and character behavior — like a **Co-Executive Producer** overseeing punch-up work.

This design ensures each scene contributes to a cohesive comedic voice and builds on established humor organically, enhancing audience engagement.


**Maybe better note how this suggestions also based off inconsistency or not**

In [ ]:
import importlib
import agents.comedy_agent
importlib.reload(agents.comedy_agent)
from agents.comedy_agent import ComedicAgent


In [ ]:
# Initialize the comedy agent with the language model client, metadata store, and scene history window
comedic_agent = ComedicAgent(
    client=client,
    vector_metadata=vector_metadata,
    num_scenes=1
)

# Evaluate Scene 2 using the comedy agent and store the evaluation results
com_context, com_is_consistent, com_analysis_text, com_recommendations, com_thoughts = evaluate_comedic_agent_scene(
    agent=comedic_agent,
    scene_description=scene_2_desc,
    scene_number=2
)

🔁 Agent will consider the last 1 scene(s) for context.

📚 Retrieving script metadata for scene(s): [1]
Scene 2 — Comedic Tone: ✅ Consistent

Comedic Tone Analysis:
3. Short Explanation: The humor in this scene is consistent with the previous scene. It continues to play on Robbie's clumsiness and poor eyesight, but also introduces a new element of humor through the forgetful elderly woman. The tone remains light-hearted and humorous.

Comedic Improvement Recommendations:
1. [Suggestion] — As Robbie is helping the elderly woman, have him use unconventional and comedic items to try to unlock the door. Perhaps he attempts to use a knitting needle, a shoe horn, or even a spatula, all of which the elderly woman keeps in her handbag "just in case." This adds a comedic visual element to the scene, while also showcasing Robbie's resourcefulness. (Justification: This can build on a running joke from previous scenes where Robbie is always seen to be resourceful, using odd items to fix problems ar

### 🌆 Environment Agent

The `EnvironmentAgent` is responsible for ensuring spatial continuity and logical scene transitions in a sitcom setting. It verifies that each new scene's location aligns naturally with prior ones and suggests immersive environmental details to enhance tone and narrative flow.

This agent operates using a **ReAct workflow**, where each step mimics a professional role in a sitcom writers' room and draws on **prior scene metadata**:

- **Think**: Logs which scene is being reviewed and determines which previous environments will provide context — like a **Writers' Assistant** organizing location continuity.
- **Act**: Analyzes the current scene’s setting and identifies key environmental elements — like a **Script Coordinator** extracting staging needs from a scene.
- **Observe**: Evaluates whether the environment transition feels natural given recent scenes — like a **Head Writer** safeguarding spatial logic and pacing.
- **Recommend**: Suggests sensory or scenic details to support the environment or smooth jarring transitions — like a **Co-Executive Producer** fine-tuning atmosphere with practical details.

Together, these steps ensure that every scene change feels intentional, immersive, and narratively cohesive — supporting both visual consistency and comedic timing.



In [ ]:
# TO DELETE
import importlib
import agents.environment_agent
importlib.reload(agents.environment_agent)
from agents.environment_agent import EnvironmentAgent


In [ ]:
# Initialize the environment agent with the language model client, metadata store, and scene history window
environment_agent = EnvironmentAgent(
    client=client,
    vector_metadata=vector_metadata,  # prior scene metadata
    num_scenes=1
)

# Evaluate Scene 2 using the environment agent and store the evaluation results
context, is_consistent, explanation, env_recommendations, env_thoughts = evaluate_environment_agent_scene(
    agent=environment_agent,
    scene_description=scene_2_desc,
    scene_number=2
)

🔁 Environment Agent will consider the last 1 scene(s) for context.

📚 Retrieving script metadata for scene(s): [1]
Scene 2 — Environment Transition: ✅ Consistent

Explanation:
Short Explanation: The transition from Robbie's apartment above the locksmith shop to Robbie's workplace is logical and believable. It is common for characters to move between their home and workplace in sitcoms. Since Robbie's apartment is above the locksmith shop, it's reasonable to assume that the locksmith shop is his workplace.
- Suggested Transition Setup: Not necessary.

Environment Detail Suggestions:
Environment Details Suggestions:
- Suggestion 1: As Robbie enters his workplace, the sound of a loud, jarring alarm bell rings out, followed by a sheepish apprentice locksmith quickly turning it off and muttering an apology. This can add a bit of humor and establish the chaotic environment of Robbie's workplace.
- Suggestion 2: An elderly woman is pacing back and forth in the waiting area, looking increasing

### 🧠 Scene Planner Agent

The `ScenePlannerAgent` acts as the **Executive Producer**, responsible for synthesizing suggestions from all other agents and outlining a coherent, engaging plan for the next scene.

Rather than simply selecting from character, comedic, and environment recommendations, this agent **distills key themes and feedback** into a unified set of creative objectives that drive the sitcom forward.

The agent uses a **ReAct-based synthesis approach**, interpreting prior agent outputs and generating new, structured goals:

- **Character Goals (2)** — derived from the `CharacterAgent`'s recommendations, focusing on growth, conflict, or relationships.
- **Comedic Goal (1)** — distilled from tone analysis and punch-up suggestions made by the `ComedicAgent`.
- **Environment Detail (1)** — extracted from the `EnvironmentAgent`'s observations and setting enhancements.
- **Creative Suggestion (1)** — an original narrative nudge that naturally integrates all the above goals into a sitcom-appropriate next step.

This agent ensures every new scene is **purposeful, tonally consistent, and narratively aligned**, helping maintain momentum and coherence throughout the episode.


In [ ]:
# TO DELETE
import importlib
import agents.scene_planner_agent
importlib.reload(agents.scene_planner_agent)
from agents.scene_planner_agent import ScenePlannerAgent

In [ ]:
# Initialize the scene planner agent with the language model client
scene_planner_agent = ScenePlannerAgent(client=client)

# Generate a plan for Scene 2 by integrating recommendations from all three agents
scene_2_plan = scene_planner_agent.plan_next_scene(
    character_recommendations=char_recommendations,
    comedic_recommendations=com_recommendations,
    environment_recommendations=env_recommendations,
    scene_number=2
)

print(scene_2_plan)

### 🎬 Generating Scene 2 Script

In [ ]:
import importlib
import screen_writing
importlib.reload(screen_writing)


<module 'screen_writing' from '/content/drive/MyDrive/Spring 2025/Gen AI with LLM/Project/utils/screen_writing.py'>

In [ ]:
# Generate the full script for Scene 2 based on the structured scene plan
scene_2_script = generate_scene(
    client=client,
    scene_plan=scene_2_plan,
    scene_number= 2
)

print(scene_2_script)

INT. ROBBIE'S LOCKSMITH WORKSHOP - DAY

Robbie, a middle-aged locksmith with an air of disarray, is explaining a lock mechanism to the elderly woman, Mrs. Perkins.

ROBBIE
...And that's how the internal mechanism works, Mrs. Perkins.

MRS. PERKINS
Fascinating, Ricky.

[Laugh Track]

ROBBIE
Robbie, Mrs. Perkins. You see, the key...

Suddenly, a comically loud drill drowns out Robbie's words. He quickly silences it with a flustered look.

ROBBIE
Sorry about that. Now, where was I? Ah, the key...

Robbie pats his pockets, searching for his master keys. He looks around, confused.

ROBBIE
Where did I put those keys?

Mrs. Perkins, amused, hands Robbie her handbag.

MRS. PERKINS
Maybe they're in here, Rodney.

[Laugh Track]

Robbie, desperate, starts rummaging through the bag. He pulls out a knitting needle, a rubber duck, and finally a spoon.

ROBBIE
Hm, well... this might just work.

He starts to fiddle with the lock using the spoon while polka music starts playing from the radio.

MRS. PE

### Adding Scene 2 to Vector Database

In [ ]:
# Store the generated Scene 2 script in the vector database for future retrieval and context
store_scene_in_vector_db(
    client=client,
    sitcom_title=sitcom_title,
    scene_script=scene_2_script,
    embedding_model=embedding_model,
    index=index,
    vector_metadata=vector_metadata
)

Total scenes stored in vector DB: 2 


Scene 1
Summary: The scene takes place in the morning in Robbie's apartment, which is situated above a locksmith shop. Robbie, a locksmith in his mid-thirties, wakes up to his alarm clock ringing. He initially misreads the time due to his poor eyesight, but after finding his glasses, he realizes he is late for a locksmith convention. In his rush to get ready, he trips over his tools and realizes he's wearing his shirt backwards. Before leaving, he grabs his locksmith ID badge, reminding himself that he needs it to enter the convention. The scene ends with a humorous note stuck to the now silent alarm clock, reminding Robbie to buy new glasses.
Characters: ['Robbie']
Location: Robbie's apartment above the locksmith shop
Recurring Joke: Robbie's poor eyesight and his clumsy, disorganized nature are recurring sources of humor in this scene.
Emotional Tone: Comedic

Scene 2
Summary: In Robbie's locksmith workshop, Robbie, a middle-aged locksmith, is e

## 🎬 Scene 3 Generation

Scene 3 (through 20) will follow the same exact structure as Scene 2

In [ ]:
# Extract the description for Scene 3 from the episode outline
scene_3_desc = extract_scene(episode_outline, 3)
print(scene_3_desc)

Scene 3: "Unexpected News" 
Robbie receives the news about his uncle's death and the inheritance of the locksmith shop; a blend of sadness and surprise.


### Character Agent

In [ ]:
# Initialize the character agent with the language model client, metadata store, and total number of scenes
character_agent = CharacterAgent(
    client=client,
    vector_metadata=vector_metadata,
    num_scenes=2
)

# Evaluate Scene 3 using the character agent and store the evaluation results
char_histories, char_is_consistent, char_explanation, char_recommendations, char_thoughts = evaluate_character_agent_scene(
    agent=character_agent,
    scene_description=scene_3_desc,
    scene_number=3
)

🔁 Agent will consider the last 2 scene(s) for context.

📚 Retrieving script metadata for scene(s): [1, 2]
Scene 3 — Consistency: ✅ Consistent

Explanation:
2. Short Explanation Why: Robbie's reaction to the unexpected news would likely be a mix of absent-mindedness and resourcefulness, consistent with his established personality traits. His patient speaking style would also be appropriate in this situation. The news about his uncle's death and the inheritance could potentially introduce a new emotional arc for Robbie, but it doesn't contradict his established character profile.

Interaction Improvement Recommendations:
1. Robbie could react to the news of his uncle's death by absentmindedly misplacing the inheritance letter and having to scramble to find it. This interaction would play off his established trait of forgetfulness from Scenes 1 and 2. It would add a comedic layer to an otherwise serious moment and be consistent with Robbie's character, who uses humor to cope with stressfu

### Comedy Agent

In [ ]:
# Initialize the comedy agent with the language model client, metadata store, and scene history window
comedic_agent = ComedicAgent(
    client=client,
    vector_metadata=vector_metadata,
    num_scenes=2
)

# Evaluate Scene 3 using the comedy agent and store the evaluation results
com_context, com_is_consistent, com_analysis_text, com_recommendations, com_thoughts = evaluate_comedic_agent_scene(
    agent=comedic_agent,
    scene_description=scene_3_desc,
    scene_number=3
)

🔁 Agent will consider the last 2 scene(s) for context.

📚 Retrieving script metadata for scene(s): [1, 2]
Scene 3 — Comedic Tone: ❌ Inconsistent

Comedic Tone Analysis:
3. Short Explanation: The previous scenes were light-hearted and slapstick, focusing on Robbie's clumsiness and poor eyesight. The new scene introduces a darker tone with the news of Robbie's uncle's death, which is a significant shift from the previous comedic tone.

Comedic Improvement Recommendations:
1. [Suggestion] - Robbie is handed a very poorly written obituary for his uncle that has glaring errors and comically misrepresents his uncle's life. This could include absurd references like his uncle being an astronaut, a professional wrestler or a sea captain, when in reality he was a humble locksmith. Robbie, with his poor eyesight, initially believes these exaggerations before realizing the silliness of it all. — (This builds on Robbie's established character trait of poor eyesight from previous scenes and introduc

### Enviroment Agent


In [ ]:
# Initialize the environment agent with the language model client, metadata store, and scene history window
environment_agent = EnvironmentAgent(
    client=client,
    vector_metadata=vector_metadata,
    num_scenes=2
)

# Evaluate Scene 3 using the environment agent and store the evaluation results
context, is_consistent, explanation, env_recommendations, env_thoughts = evaluate_environment_agent_scene(
    agent=environment_agent,
    scene_description=scene_3_desc,
    scene_number=3
)

🔁 Environment Agent will consider the last 2 scene(s) for context.

📚 Retrieving script metadata for scene(s): [1, 2]
Scene 3 — Environment Transition: ✅ Consistent

Explanation:
Short Explanation: The transition from Robbie's apartment above the locksmith shop to Robbie's Locksmith Workshop, and then to the Locksmith Shop itself is logical. It's believable that Robbie would move from his living quarters to his workshop and then to the shop, as these are all connected locations.
- Suggested Transition Setup: Not necessary in this case.

Environment Detail Suggestions:
Environment Details Suggestions:
- Suggestion 1: Robbie could be dealing with a customer who brought in an antique lock for repair. The customer insists that it's a rare, valuable piece from the 18th century, but Robbie recognizes it as a cheap replica he sold last week. This could add a funny interaction and highlight Robbie's expertise as a locksmith.
- Suggestion 2: The shop could have a lingering smell of metal and oi

### Scene Planner Agent

In [ ]:
# Initialize the scene planner agent with the language model client
scene_planner_agent = ScenePlannerAgent(client=client)

# Generate a plan for Scene 3 by integrating recommendations from all three agents
scene_3_plan = scene_planner_agent.plan_next_scene(
    character_recommendations=char_recommendations,
    comedic_recommendations=com_recommendations,
    environment_recommendations=env_recommendations,
    scene_number=3
)

print(scene_3_plan)

Scene Plan:
Character Goals:
- Goal 1: Have Robbie misplace the inheritance letter in a comedic mix-up, displaying his characteristic forgetfulness while also using humor to handle the grave news of his uncle's death.
- Goal 2: Develop a sincere moment between Robbie and Mrs. Perkins, where Robbie shares his feelings about his uncle's passing despite Mrs. Perkins consistently getting his name wrong, showcasing his patience and tenderness.

Comedic Goal:
- Goal: Incorporate physical comedy into the scene where Robbie, overwhelmed by the locksmith shop he’s inherited, knocks over a wall of keys and tries to reorganize them while dealing with customers’ puzzling requests, demonstrating his clumsiness and lack of locksmithing skills.

Environment Detail:
- Detail: The locksmith shop has a quirky charm, with an old, oversized, squeaky rotating key display that Robbie tries to navigate around, adding to the visual comedy and highlighting the shop's character.

Creative Suggestion:
- Suggesti

### Generating Scene 3 Script

In [ ]:
# Generate the full script for Scene 3 based on the structured scene plan
scene_3_script = generate_scene(
    client=client,
    scene_plan=scene_3_plan,
    scene_number= 3
)

print(scene_3_script)

INT. LOCKSMITH SHOP - DAY

Robbie is clumsily navigating around an oversized, squeaky rotating key display. He's holding a letter, which he places on the counter and promptly forgets about. 

ROBBIE:
Ah, Uncle Pete's inheritance letter. Where to put you...

He knocks over a wall of keys in the process causing a loud CLATTER.

[Laugh Track]

As Robbie scrambles to pick up the keys, the door opens and MRS. PERKINS, a sweet, elderly lady, walks in. 

MRS. PERKINS:
Good morning, Bobby!

ROBBIE:
It's Robbie, Mrs. Perkins. And good morning.

[Laugh Track]

Robbie continues to fumble with the keys, trying to rehang them, while looking for the inheritance letter.

MRS. PERKINS:
You seem a bit frazzled, dear. Everything okay?

ROBBIE:
Just dealing with some stuff, Mrs. Perkins. My Uncle Pete passed away. 

Robbie's sincerity is interrupted by a CUSTOMER who walks in with an absurd request.

CUSTOMER:
Do you have a key that opens every door in the city?

[Laugh Track]

ROBBIE:
I'm afraid not, si

### Adding Metadata to Vector Database

In [ ]:
# Store the generated Scene 3 script in the vector database for future retrieval and context
store_scene_in_vector_db(
    client=client,
    sitcom_title=sitcom_title,
    scene_script=scene_3_script,
    embedding_model=embedding_model,
    index=index,
    vector_metadata=vector_metadata
)

Total scenes stored in vector DB: 3 


Scene 1
Summary: The scene takes place in the morning in Robbie's apartment, which is situated above a locksmith shop. Robbie, a locksmith in his mid-thirties, wakes up to his alarm clock ringing. He initially misreads the time due to his poor eyesight, but after finding his glasses, he realizes he is late for a locksmith convention. In his rush to get ready, he trips over his tools and realizes he's wearing his shirt backwards. Before leaving, he grabs his locksmith ID badge, reminding himself that he needs it to enter the convention. The scene ends with a humorous note stuck to the now silent alarm clock, reminding Robbie to buy new glasses.
Characters: ['Robbie']
Location: Robbie's apartment above the locksmith shop
Recurring Joke: Robbie's poor eyesight and his clumsy, disorganized nature are recurring sources of humor in this scene.
Emotional Tone: Comedic

Scene 2
Summary: In Robbie's locksmith workshop, Robbie, a middle-aged locksmith, is e

## Scene 4 Generation

For scenes 4 and beyond, the 3 previous scenes are retreived.

In [ ]:
# Extract the description for Scene 4 from the episode outline
scene_4_desc = extract_scene(episode_outline, 4)
print(scene_4_desc)

Scene 4: "Jerry Checks In" 
Robbie's parole officer Jerry visits him at the shop, revealing his hidden love for Broadway musicals in a comical way.


### 🎭 Character Agent


In [ ]:
# Initialize the character agent with the language model client, metadata store, and total number of scenes
character_agent = CharacterAgent(
    client=client,
    vector_metadata=vector_metadata,
    num_scenes=3
)

# Evaluate Scene 4 using the character agent and store the evaluation results
char_histories, char_is_consistent, char_explanation, char_recommendations, char_thoughts = evaluate_character_agent_scene(
    agent=character_agent,
    scene_description=scene_4_desc,
    scene_number=4
)

🔁 Agent will consider the last 3 scene(s) for context.

📚 Retrieving script metadata for scene(s): [1, 2, 3]
Scene 4 — Consistency: ✅ Consistent

Explanation:
2. Short Explanation Why: Robbie's character is consistent with his established personality traits, as he is shown to be clumsy and forgetful in all three scenes. His relationship with Jerry, his parole officer, is also consistent with his friendly and casual speaking style. Jerry's character is also consistent, as his love for Broadway musicals and his comical way of revealing it aligns with his established personality trait of having a sense of humor.

Interaction Improvement Recommendations:
1. Jerry, noticing Robbie's disheveled state, asks about the reason. Robbie, in his sincere tone, shares about his Uncle Pete's passing and misplacing the inheritance letter (Scene 3), which adds a deeper dimension to their relationship. — (This interaction builds on the sincerity and seriousness Robbie showed in Scene 3 when he found out 

### 🎤 Comedic Agent


In [ ]:
# Initialize the comedy agent with the language model client, metadata store, and scene history window
comedic_agent = ComedicAgent(
    client=client,
    vector_metadata=vector_metadata,
    num_scenes=3
)

# Evaluate Scene 4 using the comedy agent and store the evaluation results
com_context, com_is_consistent, com_analysis_text, com_recommendations, com_thoughts = evaluate_comedic_agent_scene(
    agent=comedic_agent,
    scene_description=scene_4_desc,
    scene_number=4
)

🔁 Agent will consider the last 3 scene(s) for context.

📚 Retrieving script metadata for scene(s): [1, 2, 3]
Scene 4 — Comedic Tone: ✅ Consistent

Comedic Tone Analysis:
3. Short Explanation: The comedic tone of absurdity is consistent with the previous scenes. The introduction of Jerry's unexpected love for Broadway musicals adds a new layer of humor that fits well with the established comedic elements.

Comedic Improvement Recommendations:
1. [Suggestion] - As Jerry walks into the shop, have him subtly humming or quietly singing a recognizable tune from a famous Broadway musical. Robbie, confused, asks Jerry if he's singing "Memory" from Cats. Jerry, unintentionally revealing his Broadway love, enthusiastically confirms and starts discussing his favorite Cats performances — only to abruptly switch back to his stern parole officer demeanor when he realizes he's gotten off track. This sudden character shift adds humor through contrast. — (This builds on Jerry's character — tough on the

### 🌆 Environment Agent


In [ ]:
# Initialize the environment agent with the language model client, metadata store, and scene history window
environment_agent = EnvironmentAgent(
    client=client,
    vector_metadata=vector_metadata,
    num_scenes=3
)

# Evaluate Scene 4 using the environment agent and store the evaluation results
context, is_consistent, explanation, env_recommendations, env_thoughts = evaluate_environment_agent_scene(
    agent=environment_agent,
    scene_description=scene_4_desc,
    scene_number=4
)

🔁 Environment Agent will consider the last 3 scene(s) for context.

📚 Retrieving script metadata for scene(s): [1, 2, 3]
Scene 4 — Environment Transition: ✅ Consistent

Explanation:
Short Explanation: The transition from Robbie's apartment above the locksmith shop, to Robbie's Locksmith Workshop, and then to the Locksmith Shop is logical. It follows a natural progression of locations that are all connected and would be in close proximity to each other.
- Suggested Transition Setup: Not necessary.

Environment Detail Suggestions:
Environment Details Suggestions:
- [Suggestion 1] As Robbie is chatting with a customer, a loud, off-key rendition of "Phantom of the Opera" can be heard from the back room, causing Robbie to wince and apologize. Turns out, his new hire is a failed Broadway hopeful who insists on practicing during his lunch break.
- [Suggestion 2] The shop's door chime is a quirky, customized rendition of a popular Broadway show tune. Whenever someone enters or leaves, a slight

### 🧠 Scene Planner Agent


In [ ]:
# Initialize the scene planner agent with the language model client
scene_planner_agent = ScenePlannerAgent(client=client)

# Generate a plan for Scene 4 by integrating recommendations from all three agents
scene_4_plan = scene_planner_agent.plan_next_scene(
    character_recommendations=char_recommendations,
    comedic_recommendations=com_recommendations,
    environment_recommendations=env_recommendations,
    scene_number=4
)

print(scene_4_plan)

Scene Plan:
Character Goals:
- [Goal 1] Deepen the bond between Jerry and Robbie by having Robbie open up about his Uncle Pete's passing and the lost inheritance letter. This should be done in a sincere and serious manner to highlight Robbie's emotional vulnerability.
- [Goal 2] Reintroduce Robbie's comedic clumsiness from Scene 1 through a light-hearted story about a mishap at his locksmith convention, showing his ability to laugh at himself and helping to lighten the mood.

Comedic Goal:
- [Goal] Inject humor into the scene by exploiting the contrast between Jerry's stern parole officer demeanor and his hidden passion for Broadway musicals, which is accidentally revealed and then quickly repressed. This should result in a running gag throughout the rest of the episode.

Environment Detail:
- [Detail] The soundtrack to the locksmith shop is a constant, slightly out-of-tune Broadway show tune played on the shop's old radio. When the radio fizzes out or changes tune, it becomes a cue fo

### Generating Scene 4 Script

In [ ]:
# Generate the full script for Scene 4 based on the structured scene plan
scene_4_script = generate_scene(
    client=client,
    scene_plan=scene_4_plan,
    scene_number= 4
)

print(scene_4_script)

[INT. LOCKSMITH SHOP - DAY]

The BROADWAY RADIO is playing, slightly out of tune, in the background as JERRY, the stern parole officer, enters the shop.

JERRY: (in a stern voice) Robbie, we need to talk about your Uncle Pete's situation.

ROBBIE: (nods) Yeah, Jerry, I know. The lost inheritance letter... It's really tearing me up inside.

[Laugh Track]

Jerry softens a bit, showing compassion.

JERRY: I understand, Robbie. It's tough. But remember, we all have our own ways to cope.

Robbie chuckles weakly, looking back at a past incident.

ROBBIE: Like that time at the locksmith convention when I accidentally locked myself in the bathroom stall.

[Laugh Track]

Jerry can't help but laugh. The radio suddenly fizzes out, causing a brief silence.

JERRY: (laughing) You did what?

ROBBIE: (grinning) Yeah, it took three locksmiths and a janitor with a crowbar to get me out!

[Laugh Track]

Jerry, still laughing, momentarily forgets about his stern demeanor. As the radio resumes playing a B

### 🗂️ Adding Metadata to Vector Database

In [ ]:
# Store the generated Scene 4 script in the vector database for future retrieval and context
store_scene_in_vector_db(
    client=client,
    sitcom_title=sitcom_title,
    scene_script=scene_4_script,
    embedding_model=embedding_model,
    index=index,
    vector_metadata=vector_metadata
)

Total scenes stored in vector DB: 4 


Scene 1
Summary: The scene takes place in the morning in Robbie's apartment, which is situated above a locksmith shop. Robbie, a locksmith in his mid-thirties, wakes up to his alarm clock ringing. He initially misreads the time due to his poor eyesight, but after finding his glasses, he realizes he is late for a locksmith convention. In his rush to get ready, he trips over his tools and realizes he's wearing his shirt backwards. Before leaving, he grabs his locksmith ID badge, reminding himself that he needs it to enter the convention. The scene ends with a humorous note stuck to the now silent alarm clock, reminding Robbie to buy new glasses.
Characters: ['Robbie']
Location: Robbie's apartment above the locksmith shop
Recurring Joke: Robbie's poor eyesight and his clumsy, disorganized nature are recurring sources of humor in this scene.
Emotional Tone: Comedic

Scene 2
Summary: In Robbie's locksmith workshop, Robbie, a middle-aged locksmith, is e

## Scene 5 Generation

In [ ]:
# Extract the description for Scene 5 from the episode outline
scene_5_desc = extract_scene(episode_outline, 5)
print(scene_5_desc)

Scene 5: "Arrival of the Daughter" 
Melissa shows up unexpectedly, leading to an awkward reunion between the estranged father and daughter.


### 🎭 Character Agent


In [ ]:
# Initialize the character agent with the language model client, metadata store, and total number of scenes
character_agent = CharacterAgent(
    client=client,
    vector_metadata=vector_metadata,
    num_scenes=3
)

# Evaluate Scene 5 using the character agent and store the evaluation results
char_histories, char_is_consistent, char_explanation, char_recommendations, char_thoughts = evaluate_character_agent_scene(
    agent=character_agent,
    scene_description=scene_5_desc,
    scene_number=5
)

🔁 Agent will consider the last 3 scene(s) for context.

📚 Retrieving script metadata for scene(s): [2, 3, 4]
Scene 5 — Consistency: ✅ Consistent

Explanation:
2. Short Explanation Why: Based on the information provided, Melissa's bold and spontaneous arrival is consistent with her established personality. The awkward reunion with her estranged father is also logical based on their strained relationship. There are no contradictions or missing motivations evident from the last 3 scenes.

Interaction Improvement Recommendations:
1. Melissa confronts her father about their estranged relationship, expressing her feelings of abandonment and asking for an explanation. This interaction is a natural continuation from Scene 3 where the father showed signs of regret about a past decision, presumably related to Melissa's departure. 
2. The father, taken aback by Melissa's sudden arrival, attempts to avoid the confrontation by making weak jokes or changing the subject. This reaction would be in lin

### 🎤 Comedic Agent


In [ ]:
# Initialize the comedy agent with the language model client, metadata store, and scene history window
comedic_agent = ComedicAgent(
    client=client,
    vector_metadata=vector_metadata,
    num_scenes=3
)

# Evaluate Scene 5 using the comedy agent and store the evaluation results
com_context, com_is_consistent, com_analysis_text, com_recommendations, com_thoughts = evaluate_comedic_agent_scene(
    agent=comedic_agent,
    scene_description=scene_5_desc,
    scene_number=5
)

🔁 Agent will consider the last 3 scene(s) for context.

📚 Retrieving script metadata for scene(s): [2, 3, 4]
Scene 5 — Comedic Tone: ✅ Consistent

Comedic Tone Analysis:
3. Short Explanation: The awkward humor in the scene between Robbie and Melissa is consistent with the comedic tone of the previous scenes. The humor is derived from the characters' interactions and situations, similar to the previous scenes.

Comedic Improvement Recommendations:
1. [Suggestion] — As Melissa walks in the door, she trips over an oversized pair of clown shoes lying on the floor. She regains her balance and looks at her father, "Dad, why do you still have these?" He responds nonchalantly, "Oh, those? They're my lounging shoes." This plays into a running joke about the father's odd and eccentric fashion choices. It also adds an immediate layer of lightheartedness to their reunion, helping to diffuse the tension in a humorous, character-consistent way. 
2. [Suggestion] — During their conversation, Melissa a

### 🌆 Environment Agent


In [ ]:
# Initialize the environment agent with the language model client, metadata store, and scene history window
environment_agent = EnvironmentAgent(
    client=client,
    vector_metadata=vector_metadata,
    num_scenes=3
)

# Evaluate Scene 5 using the environment agent and store the evaluation results
context, is_consistent, explanation, env_recommendations, env_thoughts = evaluate_environment_agent_scene(
    agent=environment_agent,
    scene_description=scene_5_desc,
    scene_number=5
)

🔁 Environment Agent will consider the last 3 scene(s) for context.

📚 Retrieving script metadata for scene(s): [2, 3, 4]
Scene 5 — Environment Transition: ❌ Inconsistent

Explanation:
Short Explanation: There is no clear link between Robbie's locksmith workshop and his estranged father's home. The audience might be confused about why Robbie suddenly appears at his father's house without any prior context or setup.
- Suggested Transition Setup: Robbie could receive a call while at his locksmith shop, learning that his estranged father has locked himself out of his house. This would provide a logical reason for Robbie to go to his father's home, and also add an element of tension to the storyline.

Environment Detail Suggestions:
Environment Details Suggestions:
- Suggestion 1: Upon arrival, Robbie struggles to pick the lock while his father watches skeptically from the nearby window. It could be a funny moment if his father comments on Robbie’s expertise as a locksmith, despite his inab

### 🧠 Scene Planner Agent


In [ ]:
# Initialize the scene planner agent with the language model client
scene_planner_agent = ScenePlannerAgent(client=client)

# Generate a plan for Scene 5 by integrating recommendations from all three agents
scene_5_plan = scene_planner_agent.plan_next_scene(
    character_recommendations=char_recommendations,
    comedic_recommendations=com_recommendations,
    environment_recommendations=env_recommendations,
    scene_number=5
)

print(scene_5_plan)

Scene Plan:
Character Goals:
- Melissa must address her feelings of abandonment and question her father about their strained relationship, revealing a deeper layer to her character and pushing their story arc forward.
- The father should react to Melissa's confrontation with evasion tactics, such as weak jokes or subject changes, maintaining his established character trait of avoiding uncomfortable situations.

Comedic Goal:
- Incorporate a physical comedic element that plays into the father's ongoing struggle with modern technology, and a situational comedy that highlights the father's quirky fashion sense, both serving as tension-breakers amid the serious conversation.

Environment Detail:
- The room is cluttered with various sentimental and seemingly insignificant objects, offering ample opportunity for accidents and awkward moments that can be exploited for humor and character development.

Creative Suggestion:
- As Melissa attempts to confront her father about their past, the auto

### Generating Scene 5 Script

In [ ]:
# Generate the full script for Scene 5 based on the structured scene plan
scene_5_script = generate_scene(
    client=client,
    scene_plan=scene_5_plan,
    scene_number= 5
)

print(scene_5_script)

MELISSA: (looking at a photo) Dad, why don’t we ever talk about the past? You know, when mom was around.
[Laugh Track]

FATHER: (nervously laughs) Oh, you know, sweetie. I'm usually busy trying to figure out how to work this darn smartphone!
[Laugh Track]

MELISSA: (sighs) You can’t hide behind your tech problems forever, dad.

FATHER: (picks up clown shoes) Do you remember when I wore these to your school's career day? Those were the days!
[Laugh Track]

MELISSA: (frustrated) Dad, I’m serious.

(Suddenly, an automatic vacuum cleaner turns on)

FATHER: (startled) Ah! What's that? Another one of your modern contraptions? 
[Laugh Track]

MELISSA: (laughs, then becomes serious) No, Dad, it's just the vacuum cleaner. Let's get back to...

(The vacuum cleaner starts to suck up a cherished family heirloom)

MELISSA: (yells) No!

(Melissa tries to rescue the heirloom, trips over her father's clown shoes and knocks over a stack of boxes)

FATHER: (laughs) Well, that's one way to tidy up!

(A p

### 🗂️ Adding Metadata to Vector Database

In [ ]:
# Store the generated Scene 5 script in the vector database for future retrieval and context
store_scene_in_vector_db(
    client=client,
    sitcom_title=sitcom_title,
    scene_script=scene_5_script,
    embedding_model=embedding_model,
    index=index,
    vector_metadata=vector_metadata
)

Total scenes stored in vector DB: 5 


Scene 1
Summary: The scene takes place in the morning in Robbie's apartment, which is situated above a locksmith shop. Robbie, a locksmith in his mid-thirties, wakes up to his alarm clock ringing. He initially misreads the time due to his poor eyesight, but after finding his glasses, he realizes he is late for a locksmith convention. In his rush to get ready, he trips over his tools and realizes he's wearing his shirt backwards. Before leaving, he grabs his locksmith ID badge, reminding himself that he needs it to enter the convention. The scene ends with a humorous note stuck to the now silent alarm clock, reminding Robbie to buy new glasses.
Characters: ['Robbie']
Location: Robbie's apartment above the locksmith shop
Recurring Joke: Robbie's poor eyesight and his clumsy, disorganized nature are recurring sources of humor in this scene.
Emotional Tone: Comedic

Scene 2
Summary: In Robbie's locksmith workshop, Robbie, a middle-aged locksmith, is e

## Scene 6 Generation

In [ ]:
# Extract the description for Scene 6 from the episode outline
scene_6_desc = extract_scene(episode_outline, 6)
print(scene_6_desc)

Scene 6: "A Heart-to-Heart Talk" 
Robbie and Melissa talk about their past and her decision to help him run the business, providing a poignant moment amid the humor.


### 🎭 Character Agent


In [ ]:
# Initialize the character agent with the language model client, metadata store, and total number of scenes
character_agent = CharacterAgent(
    client=client,
    vector_metadata=vector_metadata,
    num_scenes=3
)

# Evaluate Scene 6 using the character agent and store the evaluation results
char_histories, char_is_consistent, char_explanation, char_recommendations, char_thoughts = evaluate_character_agent_scene(
    agent=character_agent,
    scene_description=scene_6_desc,
    scene_number=6
)

🔁 Agent will consider the last 3 scene(s) for context.

📚 Retrieving script metadata for scene(s): [3, 4, 5]
Scene 6 — Consistency: ✅ Consistent

Explanation:
2. Short Explanation Why: Robbie's sincerity and humor are consistent with his previous scenes, as he could be expected to have a serious conversation with Melissa while still maintaining his humorous side. Melissa's persistence and caring nature are also consistent, as she would likely want to help Robbie and be involved in the business. Their dialogue and actions are logical based on their established traits and relationships. There are no contradictions based on past scenes.

Interaction Improvement Recommendations:
1. Robbie clumsily spills a cup of coffee on Melissa's photo album during their heart-to-heart talk — (This interaction is consistent with Robbie's clumsiness shown in Scene 2 where he knocks over keys and misplaces Uncle Pete's inheritance letter. It would also create a humorous situation which is characteristic o

### 🎤 Comedic Agent


In [ ]:
# Initialize the comedy agent with the language model client, metadata store, and scene history window
comedic_agent = ComedicAgent(
    client=client,
    vector_metadata=vector_metadata,
    num_scenes=3

# Evaluate Scene 6 using the comedy agent and store the evaluation results
com_context, com_is_consistent, com_analysis_text, com_recommendations, com_thoughts = evaluate_comedic_agent_scene(
    agent=comedic_agent,
    scene_description=scene_6_desc,
    scene_number=6
)

🔁 Agent will consider the last 3 scene(s) for context.

📚 Retrieving script metadata for scene(s): [3, 4, 5]
Scene 6 — Comedic Tone: ✅ Consistent

Comedic Tone Analysis:
3. Short Explanation: The scene maintains a balance of humor and emotional depth, consistent with previous scenes. The humor is used to lighten the mood and provide relief during serious conversations, which is a recurring theme in the sitcom.

Comedic Improvement Recommendations:
1. [Suggestion] — As Robbie and Melissa have this heart-to-heart, let's use their surroundings to add some humor. They could be in their new office, which is still a mess from moving in. Robbie, trying to have a serious conversation, keeps getting interrupted by mundane issues. For instance, he dramatically confesses his fear of running the business alone, only for his chair to squeak loudly every time he moves. Melissa tries to suppress her laughter, saying, "Robbie, I'm trying to take you seriously, but your chair sounds like a distressed m

### 🌆 Environment Agent


In [ ]:
# Initialize the environment agent with the language model client, metadata store, and scene history window
environment_agent = EnvironmentAgent(
    client=client,
    vector_metadata=vector_metadata,
    num_scenes=3
)

# Evaluate Scene 6 using the environment agent and store the evaluation results
context, is_consistent, explanation, env_recommendations, env_thoughts = evaluate_environment_agent_scene(
    agent=environment_agent,
    scene_description=scene_6_desc,
    scene_number=6
)

🔁 Environment Agent will consider the last 3 scene(s) for context.

📚 Retrieving script metadata for scene(s): [3, 4, 5]
Scene 6 — Environment Transition: ✅ Consistent

Explanation:
Short Explanation: The transition from a locksmith shop to a business office is logical. The character could have needed the locksmith's services for their office, or the locksmith could have been called to the office for a job. The unknown location could have been a transitional scene, such as a car ride or walk from the locksmith shop to the office.
- Suggested Transition Setup: Not necessary.

Environment Detail Suggestions:
Environment Details Suggestions:
- Suggestion 1: As Robbie and Melissa are discussing business, a small paper airplane made from a financial report silently sails across the background between cubicles. Unbeknownst to them, this is a regular occurrence among the bored office employees.
- Suggestion 2: The office printer is audibly jammed. Throughout the scene, an intern in the backgr

### 🧠 Scene Planner Agent


In [ ]:
# Initialize the scene planner agent with the language model client
scene_planner_agent = ScenePlannerAgent(client=client)

# Generate a plan for Scene 6 by integrating recommendations from all three agents
scene_6_plan = scene_planner_agent.plan_next_scene(
    character_recommendations=char_recommendations,
    comedic_recommendations=com_recommendations,
    environment_recommendations=env_recommendations,
    scene_number=6
)

print(scene_6_plan)

Scene Plan:
Character Goals:
- Robbie must inadvertently cause a humorous disruption, remaining consistent with his established clumsiness, by accidentally spilling his coffee on Melissa's photo album during their serious conversation.
- Melissa will react compassionately to Robbie's blunder, utilizing this opportunity to share a humorous childhood anecdote, thereby emphasizing her caring nature and inquisitive speaking style.

Comedic Goal:
- To lighten the mood of their heart-to-heart, incorporate humor native to their characters and environment. Robbie's serious discussion should be humorously interrupted by the noisy chair, while Melissa playfully teases Robbie about his irrational fear of citrus fruits, using an orange as a comic prop.

Environment Detail:
- In the backdrop of their conversation, an unnoticed office drama unfolds, as a frustrated intern battles a rebelliously jammed printer, while other employees engage in an ongoing, silent war of paper airplane launches.

Creati

### Generating Scene 6 Script

In [ ]:
# Generate the full script for Scene 6 based on the structured scene plan
scene_6_script = generate_scene(
    client=client,
    scene_plan=scene_6_plan,
    scene_number= 6
)

print(scene_6_script)

INT. OFFICE - DAY

Robbie and Melissa sit on squeaky chairs, coffee in hand, serious conversation underway. In the background, an INTERN desperately tries to unjam a printer. Other employees launch paper airplanes at each other.

ROBBIE: 
Melissa, I just want to say-

His chair SQUEAKS loudly, interrupting him. 

[Laugh Track]

MELISSA: 
(Laughs, points at chair)
Robbie, your chair is louder than you are!

Robbie blushes, stammers, accidentally spills coffee on Melissa's photo album.

ROBBIE: 
Oh no! I'm so sorry, Melissa!

Melissa laughs, wipes coffee off the photo album.

MELISSA: 
Don't worry, Robbie. This reminds me of when I was a kid.

[Laugh Track]

Melissa shares a funny anecdote about her childhood, as Robbie listens, relieved.

MELISSA: 
And you know what’s the funniest part? The only thing we had to clean up the mess was an orange!

Melissa pulls out an orange from her bag, starts peeling it.

[Laugh Track]

Robbie gives a horrified look.

ROBBIE: 
Melissa, you know I have c

### 🗂️ Adding Metadata to Vector Database

In [ ]:
# Store the generated Scene 6 script in the vector database for future retrieval and context
store_scene_in_vector_db(
    client=client,
    sitcom_title=sitcom_title,
    scene_script=scene_6_script,
    embedding_model=embedding_model,
    index=index,
    vector_metadata=vector_metadata
)

Total scenes stored in vector DB: 6 


Scene 1
Summary: The scene takes place in the morning in Robbie's apartment, which is situated above a locksmith shop. Robbie, a locksmith in his mid-thirties, wakes up to his alarm clock ringing. He initially misreads the time due to his poor eyesight, but after finding his glasses, he realizes he is late for a locksmith convention. In his rush to get ready, he trips over his tools and realizes he's wearing his shirt backwards. Before leaving, he grabs his locksmith ID badge, reminding himself that he needs it to enter the convention. The scene ends with a humorous note stuck to the now silent alarm clock, reminding Robbie to buy new glasses.
Characters: ['Robbie']
Location: Robbie's apartment above the locksmith shop
Recurring Joke: Robbie's poor eyesight and his clumsy, disorganized nature are recurring sources of humor in this scene.
Emotional Tone: Comedic

Scene 2
Summary: In Robbie's locksmith workshop, Robbie, a middle-aged locksmith, is e

## Scene 7 Generation

In [ ]:
# Extract the description for Scene 7 from the episode outline
scene_7_desc = extract_scene(episode_outline, 7)
print(scene_7_desc)

Scene 7: "Learning the Ropes" 
Robbie teaches Melissa about locksmithing; she amazes him with her sleight of hand skills, creating a funny and surprising scene.


### 🎭 Character Agent


In [ ]:
# Initialize the character agent with the language model client, metadata store, and total number of scenes
character_agent = CharacterAgent(
    client=client,
    vector_metadata=vector_metadata,
    num_scenes=3
)

# Evaluate Scene 7 using the character agent and store the evaluation results
char_histories, char_is_consistent, char_explanation, char_recommendations, char_thoughts = evaluate_character_agent_scene(
    agent=character_agent,
    scene_description=scene_7_desc,
    scene_number=7
)

🔁 Agent will consider the last 3 scene(s) for context.

📚 Retrieving script metadata for scene(s): [4, 5, 6]
Scene 7 — Consistency: ✅ Consistent

Explanation:
1. Consistency Verdict (Yes/No)
Yes
2. Short Explanation Why (max 5 lines)
Robbie's teaching role aligns with his creative and humorous traits, and his surprise at Melissa's skills is consistent with his clumsy nature. Melissa's sleight of hand skills align with her persistent and direct personality, and her willingness to learn from Robbie reflects their friendly relationship. The scene also continues the running joke of Robbie's clumsiness and Melissa's knack for revealing hidden talents or objects.

Interaction Improvement Recommendations:
1. Robbie clumsily tries to demonstrate the art of locksmithing to Melissa but accidentally locks them both inside the workshop. They share a hearty laugh over this predicament, allowing Robbie to lighten the situation with his humor. — (This interaction fits as it is consistent with Robbie'

### 🎤 Comedic Agent


In [ ]:
# Initialize the comedy agent with the language model client, metadata store, and scene history window
comedic_agent = ComedicAgent(
    client=client,
    vector_metadata=vector_metadata,
    num_scenes=3
)

# Evaluate Scene 7 using the comedy agent and store the evaluation results
com_context, com_is_consistent, com_analysis_text, com_recommendations, com_thoughts = evaluate_comedic_agent_scene(
    agent=comedic_agent,
    scene_description=scene_7_desc,
    scene_number=7
)

🔁 Agent will consider the last 3 scene(s) for context.

📚 Retrieving script metadata for scene(s): [4, 5, 6]
Scene 7 — Comedic Tone: ✅ Consistent

Comedic Tone Analysis:
3. Short Explanation: The tone of the new scene is consistent with the previous scenes. It maintains a balance of humor and surprise, which is a recurring theme in the previous scenes. The characters continue to use humor to navigate their interactions, and unexpected elements (like Melissa's sleight of hand skills) add to the comedic surprise.

Comedic Improvement Recommendations:
1. Suggestion: As Robbie explains the locksmithing process, Melissa becomes overly enthusiastic, interrupting Robbie with outlandish hypothetical situations. For instance, she might say something like, "So, if I was trapped in an ancient tomb with only a hairpin and my wits, I could use these skills to escape, right?" Robbie responds, flustered, "Well, yeah, I suppose... but it's usually more about helping people who've locked themselves out

### 🌆 Environment Agent


In [ ]:
# Initialize the environment agent with the language model client, metadata store, and scene history window
environment_agent = EnvironmentAgent(
    client=client,
    vector_metadata=vector_metadata,
    num_scenes=3
)

# Evaluate Scene 7 using the environment agent and store the evaluation results
context, is_consistent, explanation, env_recommendations, env_thoughts = evaluate_environment_agent_scene(
    agent=environment_agent,
    scene_description=scene_7_desc,
    scene_number=7
)

🔁 Environment Agent will consider the last 3 scene(s) for context.

📚 Retrieving script metadata for scene(s): [4, 5, 6]
Scene 7 — Environment Transition: ✅ Consistent

Explanation:
Short Explanation: The transition from the office back to the locksmith shop is logical. The characters could have needed to return to the locksmith shop after their visit to the office for a variety of reasons, such as to pick up a forgotten item, to get a different key made, or to discuss further details about a lock or key issue.
- Suggested Transition Setup: Not necessary in this case.

Environment Detail Suggestions:
Environment Details Suggestions:
- [Suggestion 1]: As the characters enter the locksmith shop, the jingle of a bell above the door signals their arrival. One of the characters, unaccustomed to the sound, jumps slightly and then laughs it off, saying they need to "get that thing a volume control."
- [Suggestion 2]: The locksmith shop is filled with the smell of fresh cut metal, mixed with a

### 🧠 Scene Planner Agent


In [ ]:
# Initialize the scene planner agent with the language model client
scene_planner_agent = ScenePlannerAgent(client=client)

# Generate a plan for Scene 7 by integrating recommendations from all three agents
scene_7_plan = scene_planner_agent.plan_next_scene(
    character_recommendations=char_recommendations,
    comedic_recommendations=com_recommendations,
    environment_recommendations=env_recommendations,
    scene_number=7
)

print(scene_7_plan)

Scene Plan:
Character Goals:
- [Goal 1] Robbie should attempt to teach Melissa locksmithing in his usual clumsy manner, bumbling through the lesson before accidentally locking them both in the shop. This should highlight Robbie's endearing clumsiness and humor.
- [Goal 2] Melissa will reveal her unexpected locksmithing skills and a backstory involving a magician at a fair. This should underscore her knack for storytelling, her hidden talents, and her capacity for humor.

Comedic Goal:
- [Goal] Melissa will over-excite Robbie with her adventurous imagination, creating humorous tension as she puts an adventurous spin on the practical locksmithing skills. Additionally, she will surprise Robbie with her sleight of hand, providing the right tools at the right time, creating a funny callback to Robbie's earlier advice.

Environment Detail:
- [Detail] The shop is filled with the sound of jingling keys and the hum of machinery. Amidst the organized chaos of the shop, there's an old, slightly o

### Generating Scene 7 Script

In [ ]:
# Generate the full script for Scene 7 based on the structured scene plan
scene_7_script = generate_scene(
    client=client,
    scene_plan=scene_7_plan,
    scene_number= 7
)

print(scene_7_script)

INT. LOCKSMITH SHOP - DAY

Robbie is holding a LOCK PICK, demonstrating on a LOCK in a vice. Melissa is watching, fascinated.

ROBBIE:
And so, you see, you've got to feel the pins, like... (struggles, drops lock pick) ...like that. 

[Laugh Track]

MELISSA: (laughs)
Robbie, you're as smooth as a hedgehog on a skateboard.

ROBBIE: (grins sheepishly)
Well, the lock pick slipped, I swear.

Suddenly, the LOCK clicks open.

ROBBIE: (surprised)
Well, would you look at that. I guess I'm a natural after all.

[Laugh Track]

Robbie then turns the key in the door, locking them in.

ROBBIE: (panicked)
Uh, Melissa, we seem to be... locked in.

MELISSA: (laughs)
Don't worry, Robbie, I've got this.

Melissa pulls out a HAIRPIN and starts picking the LOCK.

ROBBIE: (amazed)
Where'd you learn to do that?

MELISSA: (grinning)
A magician at the fair, believe it or not. He taught me sleight of hand, but the skills transferred pretty well to locksmithing.

[Flashback to Melissa at a fair, being taught sle

### 🗂️ Adding Metadata to Vector Database

In [ ]:
# Store the generated Scene 7 script in the vector database for future retrieval and context
store_scene_in_vector_db(
    client=client,
    sitcom_title=sitcom_title,
    scene_script=scene_7_script,
    embedding_model=embedding_model,
    index=index,
    vector_metadata=vector_metadata
)

Total scenes stored in vector DB: 7 


Scene 1
Summary: The scene takes place in the morning in Robbie's apartment, which is situated above a locksmith shop. Robbie, a locksmith in his mid-thirties, wakes up to his alarm clock ringing. He initially misreads the time due to his poor eyesight, but after finding his glasses, he realizes he is late for a locksmith convention. In his rush to get ready, he trips over his tools and realizes he's wearing his shirt backwards. Before leaving, he grabs his locksmith ID badge, reminding himself that he needs it to enter the convention. The scene ends with a humorous note stuck to the now silent alarm clock, reminding Robbie to buy new glasses.
Characters: ['Robbie']
Location: Robbie's apartment above the locksmith shop
Recurring Joke: Robbie's poor eyesight and his clumsy, disorganized nature are recurring sources of humor in this scene.
Emotional Tone: Comedic

Scene 2
Summary: In Robbie's locksmith workshop, Robbie, a middle-aged locksmith, is e

## Scene 8 Generation

In [ ]:
# Extract the description for Scene 8 from the episode outline
scene_8_desc = extract_scene(episode_outline, 8)
print(scene_8_desc)

Scene 8: "Locksmithing Mishaps" 
Melissa's first attempt at serving a customer ends in a hilarious disaster, showing her initial struggles.


### 🎭 Character Agent


In [ ]:
# Initialize the character agent with the language model client, metadata store, and total number of scenes
character_agent = CharacterAgent(
    client=client,
    vector_metadata=vector_metadata,
    num_scenes=3
)

# Evaluate Scene 8 using the character agent and store the evaluation results
char_histories, char_is_consistent, char_explanation, char_recommendations, char_thoughts = evaluate_character_agent_scene(
    agent=character_agent,
    scene_description=scene_8_desc,
    scene_number=8
)

🔁 Agent will consider the last 3 scene(s) for context.

📚 Retrieving script metadata for scene(s): [5, 6, 7]
Scene 8 — Consistency: ✅ Consistent

Explanation:
2. Short Explanation Why: Melissa's persistence, humor, and resourcefulness are consistent with her character profile. Her struggles with serving a customer align with her previous experiences and her humorous approach to difficult situations. The customer's intrusive and unconventional behavior is also consistent with their character profile. Their interaction with Melissa could potentially lead to unusual requests, consistent with their past behavior.

Interaction Improvement Recommendations:
1. Melissa struggles to serve the customer, who requests a master key to all the doors in town. During this interaction, Melissa tries to humorously deflect the customer's absurd request by saying she only knows how to unlock doors with hairpins, referencing her resourcefulness from Scene 3. This would bring back the running joke about Mel

### 🎤 Comedic Agent


In [ ]:
# Initialize the comedy agent with the language model client, metadata store, and scene history window
comedic_agent = ComedicAgent(
    client=client,
    vector_metadata=vector_metadata,
    num_scenes=3
)

# Evaluate Scene 8 using the comedy agent and store the evaluation results
com_context, com_is_consistent, com_analysis_text, com_recommendations, com_thoughts = evaluate_comedic_agent_scene(
    agent=comedic_agent,
    scene_description=scene_8_desc,
    scene_number=8
)

🔁 Agent will consider the last 3 scene(s) for context.

📚 Retrieving script metadata for scene(s): [5, 6, 7]
Scene 8 — Comedic Tone: ✅ Consistent

Comedic Tone Analysis:
3. Short Explanation: The comedic tone in the new scene is consistent with the previous scenes. The humor is derived from physical mishaps and unexpected outcomes, similar to the previous scenes where humor was derived from physical clumsiness and unexpected events.

Comedic Improvement Recommendations:
1. [Suggestion] — As Melissa starts to serve her customer, she confidently grabs her tools saying, "I've got this." But as she bends down to work on the lock, she accidentally knocks over a display of tiny keychains, causing a cascade of jingling sounds and a small chaos around the shop. As she apologizes profusely, she tries to pick them up, only to slip on one and land on her backside. This can be a recurring joke, with Melissa trying to master her workspace clumsily, but with determination. (Justification: This ampli

### 🌆 Environment Agent


In [ ]:
# Initialize the environment agent with the language model client, metadata store, and scene history window
environment_agent = EnvironmentAgent(
    client=client,
    vector_metadata=vector_metadata,
    num_scenes=3
)

# Evaluate Scene 8 using the environment agent and store the evaluation results
context, is_consistent, explanation, env_recommendations, env_thoughts = evaluate_environment_agent_scene(
    agent=environment_agent,
    scene_description=scene_8_desc,
    scene_number=8
)

🔁 Environment Agent will consider the last 3 scene(s) for context.

📚 Retrieving script metadata for scene(s): [5, 6, 7]
Scene 8 — Environment Transition: ✅ Consistent

Explanation:
Short Explanation: The transition from the office to the locksmith shop in the previous scene sets up the current scene at the locksmith shop. It's a logical progression as the characters could have needed to get something fixed or made at the locksmith's after their time in the office.
- Suggested Transition Setup: Not necessary in this case.

Environment Detail Suggestions:
Environment Details Suggestions:
- [Suggestion 1]: In the background, the locksmith's parrot, trained to mimic sounds, starts imitating the sound of key cutting, causing some confusion and hilarity amongst the characters.
- [Suggestion 2]: The distinct smell of metal filings hangs in the air, triggering one character's obscure childhood memory about a school trip to a blacksmith shop, leading to a funny anecdote.

Environment Agent's I

### 🧠 Scene Planner Agent


In [ ]:
# Initialize the scene planner agent with the language model client
scene_planner_agent = ScenePlannerAgent(client=client)

# Generate a plan for Scene 8 by integrating recommendations from all three agents
scene_8_plan = scene_planner_agent.plan_next_scene(
    character_recommendations=char_recommendations,
    comedic_recommendations=com_recommendations,
    environment_recommendations=env_recommendations,
    scene_number=8
)

print(scene_8_plan)

Scene Plan:
Character Goals:
- [Goal 1] Melissa will try to handle the customer's bizarre request for a master key to all doors in town, attempting to use her humor and resourcefulness to navigate the situation, while also subtly showcasing her lock-picking skill.
- [Goal 2] Melissa will fumble her way to Robbie, highlighting her clumsiness and their strong bond, and seek his help in dealing with the customer's unusual demand.

Comedic Goal:
- [Goal] In the scene, Melissa will inadvertently cause a series of chaotic yet funny incidents, such as triggering a domino effect with a display of keychains and accidentally releasing a pet hamster from a hidden compartment in a door, all while trying to demonstrate her locksmithing skills.

Environment Detail:
- [Detail] The shop's atmosphere is further enlivened by the sporadic sound of key cutting echoed by the locksmith's parrot, adding to the confusion. The distinct scent of metal filings triggers Robbie's unexpected, humorous reminiscence 

### Generating Scene 8 Script

In [ ]:
# Generate the full script for Scene 8 based on the structured scene plan
scene_8_script = generate_scene(
    client=client,
    scene_plan=scene_8_plan,
    scene_number= 8
)

print(scene_8_script)

INT. LOCKSMITH SHOP - AFTERNOON

MELISSA, a spry, quick-witted locksmith, is serving a CUSTOMER, a quirky and eccentric man.

CUSTOMER
I need a key that opens all doors in town.

[Laugh Track]

MELISSA
That's a tall order, sir. I mean, I'm good, but a town master key? You've been watching too many spy movies.

CUSTOMER
Well, I believe in you.

MELISSA trips over a display of keychains, causing them to tumble like dominoes.

[Laugh Track]

MELISSA
(To herself)
Smooth, Melissa.

She gets up and walks over to ROBBIE, her best friend and colleague. He's engrossed in cutting a key, while a parrot mimics the sound.

MELISSA
Robbie, our friend here wants a master key to the town!

ROBBIE
Oh, like the one we made on that school trip to the blacksmith's shop?

[Laugh Track]

MELISSA
What? No, we made horseshoes, Robbie.

ROBBIE
Right, but remember the smell of those metal filings? Just like this place...

While they chat, MELISSA casually picks the lock on a door, which unexpectedly releases a 

### 🗂️ Adding Metadata to Vector Database

In [ ]:
# Store the generated Scene 8 script in the vector database for future retrieval and context
store_scene_in_vector_db(
    client=client,
    sitcom_title=sitcom_title,
    scene_script=scene_8_script,
    embedding_model=embedding_model,
    index=index,
    vector_metadata=vector_metadata
)

Total scenes stored in vector DB: 8 


Scene 1
Summary: The scene takes place in the morning in Robbie's apartment, which is situated above a locksmith shop. Robbie, a locksmith in his mid-thirties, wakes up to his alarm clock ringing. He initially misreads the time due to his poor eyesight, but after finding his glasses, he realizes he is late for a locksmith convention. In his rush to get ready, he trips over his tools and realizes he's wearing his shirt backwards. Before leaving, he grabs his locksmith ID badge, reminding himself that he needs it to enter the convention. The scene ends with a humorous note stuck to the now silent alarm clock, reminding Robbie to buy new glasses.
Characters: ['Robbie']
Location: Robbie's apartment above the locksmith shop
Recurring Joke: Robbie's poor eyesight and his clumsy, disorganized nature are recurring sources of humor in this scene.
Emotional Tone: Comedic

Scene 2
Summary: In Robbie's locksmith workshop, Robbie, a middle-aged locksmith, is e

## Scene 9 Generation

In [ ]:
# Extract the description for Scene 9 from the episode outline
scene_9_desc = extract_scene(episode_outline, 9)
print(scene_9_desc)

Scene 9: "Melissa's Secret Loft" 
Melissa shows Robbie her secret loft above the shop, showcasing her independent side.


### 🎭 Character Agent


In [ ]:
# Initialize the character agent with the language model client, metadata store, and total number of scenes
character_agent = CharacterAgent(
    client=client,
    vector_metadata=vector_metadata,
    num_scenes=3
)

# Evaluate Scene 9 using the character agent and store the evaluation results
char_histories, char_is_consistent, char_explanation, char_recommendations, char_thoughts = evaluate_character_agent_scene(
    agent=character_agent,
    scene_description=scene_9_desc,
    scene_number=9
)


🔁 Agent will consider the last 3 scene(s) for context.

📚 Retrieving script metadata for scene(s): [6, 7, 8]
Scene 9 — Consistency: ✅ Consistent

Explanation:
2. Short Explanation Why: Melissa's adventurous and resourceful traits are consistent with her having a secret loft. Her willingness to share it with Robbie aligns with their close relationship. Robbie's amazement at Melissa's loft would be consistent with his character, as he is often amazed by Melissa's skills and resourcefulness. His potential clumsiness in the loft could also lead to humorous situations, consistent with his established personality.

Interaction Improvement Recommendations:
1. Melissa should reveal the secret loft to Robbie with a humorous anecdote about how she discovered it, possibly involving Mr. Nibbles the hamster. — Just as she did in Scenes 1 and 3, Melissa can use her unique past experiences to lighten the situation and create a running joke about her life's oddities.
2. Robbie, in his usual clumsy fas

### 🎤 Comedic Agent


In [ ]:
# Initialize the comedy agent with the language model client, metadata store, and scene history window
comedic_agent = ComedicAgent(
    client=client,
    vector_metadata=vector_metadata,
    num_scenes=3
)

# Evaluate Scene 9 using the comedy agent and store the evaluation results
com_context, com_is_consistent, com_analysis_text, com_recommendations, com_thoughts = evaluate_comedic_agent_scene(
    agent=comedic_agent,
    scene_description=scene_9_desc,
    scene_number=9
)

🔁 Agent will consider the last 3 scene(s) for context.

📚 Retrieving script metadata for scene(s): [6, 7, 8]
Scene 9 — Comedic Tone: ✅ Consistent

Comedic Tone Analysis:
3. Short Explanation: The playful and light-hearted tone is consistent with the previous scenes. The characters continue to engage in humorous situations and banter, maintaining the comedic tone of the sitcom.

Comedic Improvement Recommendations:
1. [Suggestion] — As Melissa showcases her secret loft to Robbie, she proudly shows off her "handmade" furniture, consisting of mismatched chairs and a table with one leg shorter than the others, causing it to wobble. Melissa, totally unfazed, uses a stack of her self-help books (a running joke in previous episodes) to stabilize the table. This not only provides physical comedy but also shows Melissa's resourcefulness and reinforces her personality traits of independence and resilience.
2. [Suggestion] — During the tour, Robbie spots a corner filled with various musical instr

### 🌆 Environment Agent


In [ ]:
# Initialize the environment agent with the language model client, metadata store, and scene history window
environment_agent = EnvironmentAgent(
    client=client,
    vector_metadata=vector_metadata,
    num_scenes=3
)

# Evaluate Scene 9 using the environment agent and store the evaluation results
context, is_consistent, explanation, env_recommendations, env_thoughts = evaluate_environment_agent_scene(
    agent=environment_agent,
    scene_description=scene_9_desc,
    scene_number=9
)

🔁 Environment Agent will consider the last 3 scene(s) for context.

📚 Retrieving script metadata for scene(s): [6, 7, 8]
Scene 9 — Environment Transition: ❌ Inconsistent

Explanation:
Short Explanation: The transition from a locksmith shop to Melissa's Secret Loft is not immediately logical. There is no clear connection between these two locations that would make the transition believable.
- Suggested Transition Setup: To make this transition more believable, a scene could be added where the characters at the locksmith shop are discussing a mysterious key they've found. One of them recognizes it as a key to Melissa's loft, sparking curiosity and leading them to visit the loft.

Environment Detail Suggestions:
Environment Details Suggestions:
- Suggestion 1: As the characters enter Melissa's loft, they could be surprised by an unexpected smell, like freshly baked cookies or an overpowering perfume. This could lead to a humorous dialogue about Melissa's secret baking hobby or excessive u

### 🧠 Scene Planner Agent


In [ ]:
# Initialize the scene planner agent with the language model client
scene_planner_agent = ScenePlannerAgent(client=client)

# Generate a plan for Scene 9 by integrating recommendations from all three agents
scene_9_plan = scene_planner_agent.plan_next_scene(
    character_recommendations=char_recommendations,
    comedic_recommendations=com_recommendations,
    environment_recommendations=env_recommendations,
    scene_number=9
)

print(scene_9_plan)

Scene Plan:
Character Goals:
- Melissa to share the story of how she discovered her secret loft in a humorous way that involves Mr. Nibbles the hamster, reflecting her knack for turning odd experiences into light-hearted anecdotes.
- Robbie to unintentionally activate a hidden feature in the loft, unveiling another secret space or object, demonstrating his consistent clumsiness while also highlighting the pair's shared propensity for unexpected discoveries.

Comedic Goal:
- Incorporate a humorous exchange between Melissa and Robbie revolving around Melissa's odd collection of mismatched furniture and musical instruments, emphasizing her quirky independence and their playful dynamic.

Environment Detail:
- The loft contains an eccentric blend of mismatched items, including a wall of scented candles in bizarre fragrances like 'Blue Cheese Delight', offering a possible comedic explanation for the overpowering scent when entering the space.

Creative Suggestion:
- As Robbie accidentally tr

### Generating Scene 9 Script

In [ ]:
# Generate the full script for Scene 9 based on the structured scene plan
scene_9_script = generate_scene(
    client=client,
    scene_plan=scene_9_plan,
    scene_number= 9
)

print(scene_9_script)

INT. SECRET LOFT - DAY

Melissa and Robbie are sitting on mismatched furniture, surrounded by an array of odd musical instruments and a wall of bizarrely scented candles.

MELISSA
So there I was, Mr. Nibbles in my pocket, when I discovered this loft.

ROBBIE
Wait, back up. Mr. Nibbles was in your pocket?

[Laugh Track]

MELISSA
Yes, I’d just finished building him a mini hamster-sized zipline in my bedroom. I had to carry him up here somehow.

ROBBIE
And you didn't think to use a hamster ball?

MELISSA
And risk Mr. Nibbles not feeling the wind in his whiskers? Never!

[Laugh Track]

ROBBIE
Well, it seems you and Mr. Nibbles share a knack for ziplining into the unknown.

[Laugh Track]

Melissa picks up a tuba. 

MELISSA
You mean like how I found this tuba in a dumpster? 

[Laugh Track]

Robbie stands up to look at the wall of candles.

ROBBIE
And these candles...Blue Cheese Delight?

MELISSA
I like to think of it as a cheese plate for the nose.

[Laugh Track]

Robbie leans on a bookshelf

### 🗂️ Adding Metadata to Vector Database

In [ ]:
# Store the generated Scene 9 script in the vector database for future retrieval and context
store_scene_in_vector_db(
    client=client,
    sitcom_title=sitcom_title,
    scene_script=scene_9_script,
    embedding_model=embedding_model,
    index=index,
    vector_metadata=vector_metadata
)

Total scenes stored in vector DB: 9 


Scene 1
Summary: The scene takes place in the morning in Robbie's apartment, which is situated above a locksmith shop. Robbie, a locksmith in his mid-thirties, wakes up to his alarm clock ringing. He initially misreads the time due to his poor eyesight, but after finding his glasses, he realizes he is late for a locksmith convention. In his rush to get ready, he trips over his tools and realizes he's wearing his shirt backwards. Before leaving, he grabs his locksmith ID badge, reminding himself that he needs it to enter the convention. The scene ends with a humorous note stuck to the now silent alarm clock, reminding Robbie to buy new glasses.
Characters: ['Robbie']
Location: Robbie's apartment above the locksmith shop
Recurring Joke: Robbie's poor eyesight and his clumsy, disorganized nature are recurring sources of humor in this scene.
Emotional Tone: Comedic

Scene 2
Summary: In Robbie's locksmith workshop, Robbie, a middle-aged locksmith, is e

## Scene 10 Generation

In [ ]:
# Extract the description for Scene 10 from the episode outline
scene_10_desc = extract_scene(episode_outline, 10)
print(scene_10_desc)

Scene 10: "Evening at the Shop" 
Robbie and Melissa spend an evening at the shop, bonding over shared stories and laughs.


### 🎭 Character Agent


In [ ]:
# Initialize the character agent with the language model client, metadata store, and total number of scenes
character_agent = CharacterAgent(
    client=client,
    vector_metadata=vector_metadata,
    num_scenes=3
)

# Evaluate Scene 10 using the character agent and store the evaluation results
char_histories, char_is_consistent, char_explanation, char_recommendations, char_thoughts = evaluate_character_agent_scene(
    agent=character_agent,
    scene_description=scene_10_desc,
    scene_number=10
)

🔁 Agent will consider the last 3 scene(s) for context.

📚 Retrieving script metadata for scene(s): [7, 8, 9]
Scene 10 — Consistency: ✅ Consistent

Explanation:
2. Short Explanation Why: The planned scene "Evening at the Shop" aligns with the established character profiles of Robbie and Melissa. Robbie's engaging conversational style and humorous personality are consistent with bonding over shared stories and laughs. Similarly, Melissa's witty speaking style and her enjoyment of sharing stories about her life also align with the planned scene. Their close relationship, as established in previous scenes, supports the idea of them spending an evening together at the shop.

Interaction Improvement Recommendations:
1. Robbie challenges Melissa to a friendly lock-picking competition using a particularly complex lock he's been working on. — Robbie's playful and competitive nature was first shown in Scene 1 when he challenged Melissa to unlock the door. This interaction builds on that while al

### 🎤 Comedic Agent


In [ ]:
# Initialize the comedy agent with the language model client, metadata store, and scene history window
comedic_agent = ComedicAgent(
    client=client,
    vector_metadata=vector_metadata,
    num_scenes=3
)

# Evaluate Scene 10 using the comedy agent and store the evaluation results
com_context, com_is_consistent, com_analysis_text, com_recommendations, com_thoughts = evaluate_comedic_agent_scene(
    agent=comedic_agent,
    scene_description=scene_10_desc,
    scene_number=10
)

🔁 Agent will consider the last 3 scene(s) for context.

📚 Retrieving script metadata for scene(s): [7, 8, 9]
Scene 10 — Comedic Tone: ✅ Consistent

Comedic Tone Analysis:
3. Short Explanation: The tone of the scene is consistent with the previous scenes, maintaining a light-hearted and playful atmosphere. The characters continue to bond over shared stories and laughs, which is in line with their established dynamic.

Comedic Improvement Recommendations:
1. Suggestion: While sharing stories, have Robbie tell a story about a ridiculous customer request, only for Melissa to realize that she was that customer. She tries to play it off like it was a joke, but Robbie isn't convinced. This could lead to a light, humorous conflict and give Melissa an opportunity to show her quick wit. — (This could build on a running joke about Robbie's encounters with eccentric customers and add a surprising twist by making Melissa one of them. It also reinforces the dynamic between Robbie and Melissa, where 

### 🌆 Environment Agent


In [ ]:
# Initialize the environment agent with the language model client, metadata store, and scene history window
environment_agent = EnvironmentAgent(
    client=client,
    vector_metadata=vector_metadata,
    num_scenes=3
)

# Evaluate Scene 10 using the environment agent and store the evaluation results
context, is_consistent, explanation, env_recommendations, env_thoughts = evaluate_environment_agent_scene(
    agent=environment_agent,
    scene_description=scene_10_desc,
    scene_number=10
)

🔁 Environment Agent will consider the last 3 scene(s) for context.

📚 Retrieving script metadata for scene(s): [7, 8, 9]
Scene 10 — Environment Transition: ✅ Consistent

Explanation:
Short Explanation: The transition from a Secret Loft to a Shop is logical and believable. The characters could have left the Secret Loft and gone to a Shop for any number of reasons, such as to buy supplies or to meet someone. The Shop could also be a different part of the Locksmith Shop, which would make the transition even more seamless.
- Suggested Transition Setup: Not necessary in this case.

Environment Detail Suggestions:
Environment Details Suggestions:
- [Suggestion 1] As one of the characters is browsing through the shop, they come across a "singing fish" wall plaque, which unexpectedly starts singing a catchy but annoying tune. They jump in surprise, which adds a touch of humor to the scene.
- [Suggestion 2] The shopkeeper is an eccentric individual who is constantly trying to sell a clearly out

### 🧠 Scene Planner Agent



In [ ]:
# Initialize the scene planner agent with the language model client
scene_planner_agent = ScenePlannerAgent(client=client)

# Generate a plan for Scene 10 by integrating recommendations from all three agents
scene_10_plan = scene_planner_agent.plan_next_scene(
    character_recommendations=char_recommendations,
    comedic_recommendations=com_recommendations,
    environment_recommendations=env_recommendations,
    scene_number=10
)

print(scene_10_plan)

Scene Plan:
Character Goals:
- [Goal 1] During a playful rivalry, Robbie unveils a complex lock he's been working on for a lock-picking competition against Melissa, demonstrating his competitive side and interest in intricate lock designs.  
- [Goal 2] Melissa unintentionally uncovers another secret compartment within an antique clock in the shop, revealing her knack for finding hidden secrets and igniting Robbie's adventurous spirit for another thrilling endeavor.

Comedic Goal:
- [Goal] Weave in comedic moments where Robbie narrates a tale of an absurd customer request, and Melissa unexpectedly turns out to be that bizarre customer. Generate laughter when Melissa attempts to disguise her embarrassment as a joke, and Robbie remains suspicious yet amused.

Environment Detail:
- [Detail] The atmosphere of the shop is enhanced by an unusual "talking parrot" toy that randomly squawks humorous phrases, startling the characters and adding a layer of comedy to the scene.

Creative Suggestion

### Generating Scene 10 Script

In [ ]:
# Generate the full script for Scene 10 based on the structured scene plan
scene_10_script = generate_scene(
    client=client,
    scene_plan=scene_10_plan,
    scene_number=10
)

print(scene_10_script)

INT. OLD ANTIQUE SHOP - DAY

Robbie, a quirky locksmith, is hunched over a complex lock on a workbench. Melissa, a lively woman with a knack for finding hidden secrets, is examining an antique clock nearby.

ROBBIE:
(looking up from the lock)
You know, Melissa, I once had a customer who wanted a lock for their antique clock. They insisted on a secret compartment too. Can you believe that?

[Laugh Track]

MELISSA:
(mischievously)
Well, Robbie, that doesn't sound so strange to me.

Robbie looks at Melissa, puzzled. The parrot toy squawks from its perch.

PARROT TOY:
Crackers! Who's got the crackers?

[Laugh Track]

Robbie shrugs and returns to his lock. Melissa continues with the clock. Suddenly, the clock chimes, and a secret compartment springs open.

MELISSA:
(surprised)
Robbie, look! Another secret compartment!

Robbie rushes over, his competitive spirit ignited.

ROBBIE:
(excited)
Wow, Melissa, you've got a real knack for this!

They excitedly investigate the compartment. The parrot

### 🗂️ Adding Metadata to Vector Database


In [ ]:
# Store the generated Scene 10 script in the vector database for future retrieval and context
store_scene_in_vector_db(
    client=client,
    sitcom_title=sitcom_title,
    scene_script=scene_10_script,
    embedding_model=embedding_model,
    index=index,
    vector_metadata=vector_metadata
)

Total scenes stored in vector DB: 10 


Scene 1
Summary: The scene takes place in the morning in Robbie's apartment, which is situated above a locksmith shop. Robbie, a locksmith in his mid-thirties, wakes up to his alarm clock ringing. He initially misreads the time due to his poor eyesight, but after finding his glasses, he realizes he is late for a locksmith convention. In his rush to get ready, he trips over his tools and realizes he's wearing his shirt backwards. Before leaving, he grabs his locksmith ID badge, reminding himself that he needs it to enter the convention. The scene ends with a humorous note stuck to the now silent alarm clock, reminding Robbie to buy new glasses.
Characters: ['Robbie']
Location: Robbie's apartment above the locksmith shop
Recurring Joke: Robbie's poor eyesight and his clumsy, disorganized nature are recurring sources of humor in this scene.
Emotional Tone: Comedic

Scene 2
Summary: In Robbie's locksmith workshop, Robbie, a middle-aged locksmith, is 

## 🎬 Scene 11 Generation


In [ ]:
# Extract the description for Scene 11 from the episode outline
scene_11_desc = extract_scene(episode_outline, 11)
print(scene_11_desc)

### 🎭 Character Agent


In [ ]:
# Re-initialize the character agent with the updated vector metadata
character_agent = CharacterAgent(
    client=client,
    vector_metadata=vector_metadata,
    num_scenes=3
)

# Evaluate Scene 11 using the character agent and store the evaluation results
char_histories, char_is_consistent, char_explanation, char_recommendations, char_thoughts = evaluate_character_agent_scene(
    agent=character_agent,
    scene_description=scene_11_desc,
    scene_number=11
)


### 🎤 Comedic Agent


In [ ]:
# Re-initialize the comedy agent with the updated vector metadata
comedic_agent = ComedicAgent(
    client=client,
    vector_metadata=vector_metadata,
    num_scenes=3
)

# Evaluate Scene 11 using the comedy agent and store the evaluation results
com_context, com_is_consistent, com_analysis_text, com_recommendations, com_thoughts = evaluate_comedic_agent_scene(
    agent=comedic_agent,
    scene_description=scene_11_desc,
    scene_number=11
)

### 🌆 Environment Agent


In [ ]:
# Re-initialize the environment agent with the updated vector metadata
environment_agent = EnvironmentAgent(
    client=client,
    vector_metadata=vector_metadata,
    num_scenes=3
)

# Evaluate Scene 11 using the environment agent and store the evaluation results
context, is_consistent, explanation, env_recommendations, env_thoughts = evaluate_environment_agent_scene(
    agent=environment_agent,
    scene_description=scene_11_desc,
    scene_number=11
)

### 🧠 Scene Planner Agent


In [ ]:
# Generate a plan for Scene 11 by integrating recommendations from all three agents
scene_11_plan = scene_planner_agent.plan_next_scene(
    character_recommendations=char_recommendations,
    comedic_recommendations=com_recommendations,
    environment_recommendations=env_recommendations,
    scene_number=11
)

print(scene_11_plan)

### ✍️ Generating Scene 11 Script

In [ ]:
# Generate the full script for Scene 11 based on the structured scene plan
scene_11_script = generate_scene(
    client=client,
    scene_plan=scene_11_plan,
    scene_number=11
)

print(scene_11_script)

### 🗂️ Adding Metadata to Vector Database


In [ ]:
# Store the generated Scene 11 script in the vector database for future retrieval and context
store_scene_in_vector_db(
    client=client,
    sitcom_title=sitcom_title,
    scene_script=scene_11_script,
    embedding_model=embedding_model,
    index=index,
    vector_metadata=vector_metadata
)

## 🎬 Scene 12 Generation


In [ ]:
# Extract the description for Scene 12 from the episode outline
scene_12_desc = extract_scene(episode_outline, 12)
print(scene_12_desc)

### 🎭 Character Agent


In [ ]:
# Re-initialize the character agent with the updated vector metadata
character_agent = CharacterAgent(
    client=client,
    vector_metadata=vector_metadata,
    num_scenes=3
)

# Evaluate Scene 12 using the character agent and store the evaluation results
char_histories, char_is_consistent, char_explanation, char_recommendations, char_thoughts = evaluate_character_agent_scene(
    agent=character_agent,
    scene_description=scene_12_desc,
    scene_number=12
)

### 🎤 Comedic Agent


In [ ]:
# Re-initialize the comedy agent with the updated vector metadata
comedic_agent = ComedicAgent(
    client=client,
    vector_metadata=vector_metadata,
    num_scenes=3
)

# Evaluate Scene 12 using the comedy agent and store the evaluation results
com_context, com_is_consistent, com_analysis_text, com_recommendations, com_thoughts = evaluate_comedic_agent_scene(
    agent=comedic_agent,
    scene_description=scene_12_desc,
    scene_number=12
)

### 🌆 Environment Agent


In [ ]:
# Re-initialize the environment agent with the updated vector metadata
environment_agent = EnvironmentAgent(
    client=client,
    vector_metadata=vector_metadata,
    num_scenes=3
)

# Evaluate Scene 12 using the environment agent and store the evaluation results
context, is_consistent, explanation, env_recommendations, env_thoughts = evaluate_environment_agent_scene(
    agent=environment_agent,
    scene_description=scene_12_desc,
    scene_number=12
)

### 🧠 Scene Planner Agent


In [ ]:
# Generate a plan for Scene 12 by integrating recommendations from all three agents
scene_12_plan = scene_planner_agent.plan_next_scene(
    character_recommendations=char_recommendations,
    comedic_recommendations=com_recommendations,
    environment_recommendations=env_recommendations,
    scene_number=12
)

print(scene_12_plan)

### ✍️ Generating Scene 12 Script

In [ ]:
# Generate the full script for Scene 12 based on the structured scene plan
scene_12_script = generate_scene(
    client=client,
    scene_plan=scene_12_plan,
    scene_number=12
)

print(scene_12_script)

### 🗂️ Adding Metadata to Vector Database


In [ ]:
# Store the generated Scene 12 script in the vector database for future retrieval and context
store_scene_in_vector_db(
    client=client,
    sitcom_title=sitcom_title,
    scene_script=scene_12_script,
    embedding_model=embedding_model,
    index=index,
    vector_metadata=vector_metadata
)

## 🎬 Scene 13 Generation


In [ ]:
# Extract the description for Scene 13 from the episode outline
scene_13_desc = extract_scene(episode_outline, 13)
print(scene_13_desc)

### 🎭 Character Agent


In [ ]:
# Re-initialize the character agent with the updated vector metadata
character_agent = CharacterAgent(
    client=client,
    vector_metadata=vector_metadata,
    num_scenes=3
)

# Evaluate Scene 13 using the character agent and store the evaluation results
char_histories, char_is_consistent, char_explanation, char_recommendations, char_thoughts = evaluate_character_agent_scene(
    agent=character_agent,
    scene_description=scene_13_desc,
    scene_number=13
)

### 🎤 Comedic Agent


In [ ]:
# Re-initialize the comedy agent with the updated vector metadata
comedic_agent = ComedicAgent(
    client=client,
    vector_metadata=vector_metadata,
    num_scenes=3
)

# Evaluate Scene 13 using the comedy agent and store the evaluation results
com_context, com_is_consistent, com_analysis_text, com_recommendations, com_thoughts = evaluate_comedic_agent_scene(
    agent=comedic_agent,
    scene_description=scene_13_desc,
    scene_number=13
)

### 🌆 Environment Agent


In [ ]:
# Re-initialize the environment agent with the updated vector metadata
environment_agent = EnvironmentAgent(
    client=client,
    vector_metadata=vector_metadata,
    num_scenes=3
)

# Evaluate Scene 13 using the environment agent and store the evaluation results
context, is_consistent, explanation, env_recommendations, env_thoughts = evaluate_environment_agent_scene(
    agent=environment_agent,
    scene_description=scene_13_desc,
    scene_number=13
)

### 🧠 Scene Planner Agent


In [ ]:
# Generate a plan for Scene 13 by integrating recommendations from all three agents
scene_13_plan = scene_planner_agent.plan_next_scene(
    character_recommendations=char_recommendations,
    comedic_recommendations=com_recommendations,
    environment_recommendations=env_recommendations,
    scene_number=13
)

print(scene_13_plan)

### ✍️ Generating Scene 13 Script

In [ ]:
# Generate the full script for Scene 13 based on the structured scene plan
scene_13_script = generate_scene(
    client=client,
    scene_plan=scene_13_plan,
    scene_number=13
)

print(scene_13_script)

### 🗂️ Adding Metadata to Vector Database


In [ ]:
# Store the generated Scene 13 script in the vector database for future retrieval and context
store_scene_in_vector_db(
    client=client,
    sitcom_title=sitcom_title,
    scene_script=scene_13_script,
    embedding_model=embedding_model,
    index=index,
    vector_metadata=vector_metadata
)

## 🎬 Scene 14 Generation


In [ ]:
# Extract the description for Scene 14 from the episode outline
scene_14_desc = extract_scene(episode_outline, 14)
print(scene_14_desc)

### 🎭 Character Agent


In [ ]:
# Re-initialize the character agent with the updated vector metadata
character_agent = CharacterAgent(
    client=client,
    vector_metadata=vector_metadata,
    num_scenes=3
)

# Evaluate Scene 14 using the character agent and store the evaluation results
char_histories, char_is_consistent, char_explanation, char_recommendations, char_thoughts = evaluate_character_agent_scene(
    agent=character_agent,
    scene_description=scene_14_desc,
    scene_number=14
)

### 🎤 Comedic Agent


In [ ]:
# Re-initialize the comedy agent with the updated vector metadata
comedic_agent = ComedicAgent(
    client=client,
    vector_metadata=vector_metadata,
    num_scenes=3
)

# Evaluate Scene 14 using the comedy agent and store the evaluation results
com_context, com_is_consistent, com_analysis_text, com_recommendations, com_thoughts = evaluate_comedic_agent_scene(
    agent=comedic_agent,
    scene_description=scene_14_desc,
    scene_number=14
)

### 🌆 Environment Agent


In [ ]:
# Re-initialize the environment agent with the updated vector metadata
environment_agent = EnvironmentAgent(
    client=client,
    vector_metadata=vector_metadata,
    num_scenes=3
)

# Evaluate Scene 14 using the environment agent and store the evaluation results
context, is_consistent, explanation, env_recommendations, env_thoughts = evaluate_environment_agent_scene(
    agent=environment_agent,
    scene_description=scene_14_desc,
    scene_number=14
)

### 🧠 Scene Planner Agent


In [ ]:
# Generate a plan for Scene 14 by integrating recommendations from all three agents
scene_14_plan = scene_planner_agent.plan_next_scene(
    character_recommendations=char_recommendations,
    comedic_recommendations=com_recommendations,
    environment_recommendations=env_recommendations,
    scene_number=14
)

print(scene_14_plan)

### ✍️ Generating Scene 14 Script

In [ ]:
# Generate the full script for Scene 14 based on the structured scene plan
scene_14_script = generate_scene(
    client=client,
    scene_plan=scene_14_plan,
    scene_number=14
)

print(scene_14_script)

### 🗂️ Adding Metadata to Vector Database


In [ ]:
# Store the generated Scene 14 script in the vector database for future retrieval and context
store_scene_in_vector_db(
    client=client,
    sitcom_title=sitcom_title,
    scene_script=scene_14_script,
    embedding_model=embedding_model,
    index=index,
    vector_metadata=vector_metadata
)

## 🎬 Scene 15 Generation


In [ ]:
# Extract the description for Scene 15 from the episode outline
scene_15_desc = extract_scene(episode_outline, 15)
print(scene_15_desc)

### 🎭 Character Agent


In [ ]:
# Re-initialize the character agent with the updated vector metadata
character_agent = CharacterAgent(
    client=client,
    vector_metadata=vector_metadata,
    num_scenes=3
)

# Evaluate Scene 15 using the character agent and store the evaluation results
char_histories, char_is_consistent, char_explanation, char_recommendations, char_thoughts = evaluate_character_agent_scene(
    agent=character_agent,
    scene_description=scene_15_desc,
    scene_number=15
)

### 🎤 Comedic Agent


In [ ]:
# Re-initialize the comedy agent with the updated vector metadata
comedic_agent = ComedicAgent(
    client=client,
    vector_metadata=vector_metadata,
    num_scenes=3
)

# Evaluate Scene 15 using the comedy agent and store the evaluation results
com_context, com_is_consistent, com_analysis_text, com_recommendations, com_thoughts = evaluate_comedic_agent_scene(
    agent=comedic_agent,
    scene_description=scene_15_desc,
    scene_number=15
)

### 🌆 Environment Agent


In [ ]:
# Re-initialize the environment agent with the updated vector metadata
environment_agent = EnvironmentAgent(
    client=client,
    vector_metadata=vector_metadata,
    num_scenes=3
)

# Evaluate Scene 15 using the environment agent and store the evaluation results
context, is_consistent, explanation, env_recommendations, env_thoughts = evaluate_environment_agent_scene(
    agent=environment_agent,
    scene_description=scene_15_desc,
    scene_number=15
)

### 🧠 Scene Planner Agent


In [ ]:
# Generate a plan for Scene 15 by integrating recommendations from all three agents
scene_15_plan = scene_planner_agent.plan_next_scene(
    character_recommendations=char_recommendations,
    comedic_recommendations=com_recommendations,
    environment_recommendations=env_recommendations,
    scene_number=15
)

print(scene_15_plan)

### ✍️ Generating Scene 15 Script

In [ ]:
# Generate the full script for Scene 15 based on the structured scene plan
scene_15_script = generate_scene(
    client=client,
    scene_plan=scene_15_plan,
    scene_number=15
)

print(scene_15_script)

### 🗂️ Adding Metadata to Vector Database


In [ ]:
# Store the generated Scene 15 script in the vector database for future retrieval and context
store_scene_in_vector_db(
    client=client,
    sitcom_title=sitcom_title,
    scene_script=scene_15_script,
    embedding_model=embedding_model,
    index=index,
    vector_metadata=vector_metadata
)

## 🎬 Scene 16 Generation


In [ ]:
# Extract the description for Scene 16 from the episode outline
scene_16_desc = extract_scene(episode_outline, 16)
print(scene_16_desc)

### 🎭 Character Agent


In [ ]:
# Re-initialize the character agent with the updated vector metadata
character_agent = CharacterAgent(
    client=client,
    vector_metadata=vector_metadata,
    num_scenes=3
)

# Evaluate Scene 16 using the character agent and store the evaluation results
char_histories, char_is_consistent, char_explanation, char_recommendations, char_thoughts = evaluate_character_agent_scene(
    agent=character_agent,
    scene_description=scene_16_desc,
    scene_number=16
)

### 🎤 Comedic Agent


In [ ]:
# Re-initialize the comedy agent with the updated vector metadata
comedic_agent = ComedicAgent(
    client=client,
    vector_metadata=vector_metadata,
    num_scenes=3
)

# Evaluate Scene 16 using the comedy agent and store the evaluation results
com_context, com_is_consistent, com_analysis_text, com_recommendations, com_thoughts = evaluate_comedic_agent_scene(
    agent=comedic_agent,
    scene_description=scene_16_desc,
    scene_number=16
)

### 🌆 Environment Agent


In [ ]:
# Re-initialize the environment agent with the updated vector metadata
environment_agent = EnvironmentAgent(
    client=client,
    vector_metadata=vector_metadata,
    num_scenes=3
)

# Evaluate Scene 16 using the environment agent and store the evaluation results
context, is_consistent, explanation, env_recommendations, env_thoughts = evaluate_environment_agent_scene(
    agent=environment_agent,
    scene_description=scene_16_desc,
    scene_number=16
)

### 🧠 Scene Planner Agent


In [ ]:
# Generate a plan for Scene 16 by integrating recommendations from all three agents
scene_16_plan = scene_planner_agent.plan_next_scene(
    character_recommendations=char_recommendations,
    comedic_recommendations=com_recommendations,
    environment_recommendations=env_recommendations,
    scene_number=16
)

print(scene_16_plan)

### ✍️ Generating Scene 16 Script

In [ ]:
# Generate the full script for Scene 16 based on the structured scene plan
scene_16_script = generate_scene(
    client=client,
    scene_plan=scene_16_plan,
    scene_number=16
)

print(scene_16_script)

### 🗂️ Adding Metadata to Vector Database


In [ ]:
# Store the generated Scene 16 script in the vector database for future retrieval and context
store_scene_in_vector_db(
    client=client,
    sitcom_title=sitcom_title,
    scene_script=scene_16_script,
    embedding_model=embedding_model,
    index=index,
    vector_metadata=vector_metadata
)

## 🎬 Scene 17 Generation


In [ ]:
# Extract the description for Scene 17 from the episode outline
scene_17_desc = extract_scene(episode_outline, 17)
print(scene_17_desc)

### 🎭 Character Agent


In [ ]:
# Re-initialize the character agent with the updated vector metadata
character_agent = CharacterAgent(
    client=client,
    vector_metadata=vector_metadata,
    num_scenes=3
)

# Evaluate Scene 17 using the character agent and store the evaluation results
char_histories, char_is_consistent, char_explanation, char_recommendations, char_thoughts = evaluate_character_agent_scene(
    agent=character_agent,
    scene_description=scene_17_desc,
    scene_number=17
)

### 🎤 Comedic Agent


In [ ]:
# Re-initialize the comedy agent with the updated vector metadata
comedic_agent = ComedicAgent(
    client=client,
    vector_metadata=vector_metadata,
    num_scenes=3
)

# Evaluate Scene 17 using the comedy agent and store the evaluation results
com_context, com_is_consistent, com_analysis_text, com_recommendations, com_thoughts = evaluate_comedic_agent_scene(
    agent=comedic_agent,
    scene_description=scene_17_desc,
    scene_number=17
)

### 🌆 Environment Agent


In [ ]:
# Re-initialize the environment agent with the updated vector metadata
environment_agent = EnvironmentAgent(
    client=client,
    vector_metadata=vector_metadata,
    num_scenes=3
)

# Evaluate Scene 17 using the environment agent and store the evaluation results
context, is_consistent, explanation, env_recommendations, env_thoughts = evaluate_environment_agent_scene(
    agent=environment_agent,
    scene_description=scene_17_desc,
    scene_number=17
)

### 🧠 Scene Planner Agent


In [ ]:
# Generate a plan for Scene 17 by integrating recommendations from all three agents
scene_17_plan = scene_planner_agent.plan_next_scene(
    character_recommendations=char_recommendations,
    comedic_recommendations=com_recommendations,
    environment_recommendations=env_recommendations,
    scene_number=17
)

print(scene_17_plan)

### ✍️ Generating Scene 17 Script

In [ ]:
# Generate the full script for Scene 17 based on the structured scene plan
scene_17_script = generate_scene(
    client=client,
    scene_plan=scene_17_plan,
    scene_number=17
)

print(scene_17_script)

### 🗂️ Adding Metadata to Vector Database


In [ ]:
# Store the generated Scene 17 script in the vector database for future retrieval and context
store_scene_in_vector_db(
    client=client,
    sitcom_title=sitcom_title,
    scene_script=scene_17_script,
    embedding_model=embedding_model,
    index=index,
    vector_metadata=vector_metadata
)

## 🎬 Scene 18 Generation


In [ ]:
# Extract the description for Scene 18 from the episode outline
scene_18_desc = extract_scene(episode_outline, 18)
print(scene_18_desc)

### 🎭 Character Agent


In [ ]:
# Re-initialize the character agent with the updated vector metadata
character_agent = CharacterAgent(
    client=client,
    vector_metadata=vector_metadata,
    num_scenes=3
)

# Evaluate Scene 18 using the character agent and store the evaluation results
char_histories, char_is_consistent, char_explanation, char_recommendations, char_thoughts = evaluate_character_agent_scene(
    agent=character_agent,
    scene_description=scene_18_desc,
    scene_number=18
)

### 🎤 Comedic Agent


In [ ]:
# Re-initialize the comedy agent with the updated vector metadata
comedic_agent = ComedicAgent(
    client=client,
    vector_metadata=vector_metadata,
    num_scenes=3
)

# Evaluate Scene 18 using the comedy agent and store the evaluation results
com_context, com_is_consistent, com_analysis_text, com_recommendations, com_thoughts = evaluate_comedic_agent_scene(
    agent=comedic_agent,
    scene_description=scene_18_desc,
    scene_number=18
)

### 🌆 Environment Agent


In [ ]:
# Re-initialize the environment agent with the updated vector metadata
environment_agent = EnvironmentAgent(
    client=client,
    vector_metadata=vector_metadata,
    num_scenes=3
)

# Evaluate Scene 18 using the environment agent and store the evaluation results
context, is_consistent, explanation, env_recommendations, env_thoughts = evaluate_environment_agent_scene(
    agent=environment_agent,
    scene_description=scene_18_desc,
    scene_number=18
)

### 🧠 Scene Planner Agent


In [ ]:
# Generate a plan for Scene 18 by integrating recommendations from all three agents
scene_18_plan = scene_planner_agent.plan_next_scene(
    character_recommendations=char_recommendations,
    comedic_recommendations=com_recommendations,
    environment_recommendations=env_recommendations,
    scene_number=18
)

print(scene_18_plan)

### ✍️ Generating Scene 18 Script

In [ ]:
# Generate the full script for Scene 18 based on the structured scene plan
scene_18_script = generate_scene(
    client=client,
    scene_plan=scene_18_plan,
    scene_number=18
)

print(scene_18_script)

### 🗂️ Adding Metadata to Vector Database


In [ ]:
# Store the generated Scene 18 script in the vector database for future retrieval and context
store_scene_in_vector_db(
    client=client,
    sitcom_title=sitcom_title,
    scene_script=scene_18_script,
    embedding_model=embedding_model,
    index=index,
    vector_metadata=vector_metadata
)

## 🎬 Scene 19 Generation


In [ ]:
# Extract the description for Scene 19 from the episode outline
scene_19_desc = extract_scene(episode_outline, 19)
print(scene_19_desc)

### 🎭 Character Agent


In [ ]:
# Re-initialize the character agent with the updated vector metadata
character_agent = CharacterAgent(
    client=client,
    vector_metadata=vector_metadata,
    num_scenes=3
)

# Evaluate Scene 19 using the character agent and store the evaluation results
char_histories, char_is_consistent, char_explanation, char_recommendations, char_thoughts = evaluate_character_agent_scene(
    agent=character_agent,
    scene_description=scene_19_desc,
    scene_number=19
)

### 🎤 Comedic Agent


In [ ]:
# Re-initialize the comedy agent with the updated vector metadata
comedic_agent = ComedicAgent(
    client=client,
    vector_metadata=vector_metadata,
    num_scenes=3
)

# Evaluate Scene 19 using the comedy agent and store the evaluation results
com_context, com_is_consistent, com_analysis_text, com_recommendations, com_thoughts = evaluate_comedic_agent_scene(
    agent=comedic_agent,
    scene_description=scene_19_desc,
    scene_number=19
)

### 🌆 Environment Agent


In [ ]:
# Re-initialize the environment agent with the updated vector metadata
environment_agent = EnvironmentAgent(
    client=client,
    vector_metadata=vector_metadata,
    num_scenes=3
)

# Evaluate Scene 19 using the environment agent and store the evaluation results
context, is_consistent, explanation, env_recommendations, env_thoughts = evaluate_environment_agent_scene(
    agent=environment_agent,
    scene_description=scene_19_desc,
    scene_number=19
)

### 🧠 Scene Planner Agent


In [ ]:
# Generate a plan for Scene 19 by integrating recommendations from all three agents
scene_19_plan = scene_planner_agent.plan_next_scene(
    character_recommendations=char_recommendations,
    comedic_recommendations=com_recommendations,
    environment_recommendations=env_recommendations,
    scene_number=19
)

print(scene_19_plan)

### ✍️ Generating Scene 19 Script

In [ ]:
# Generate the full script for Scene 19 based on the structured scene plan
scene_19_script = generate_scene(
    client=client,
    scene_plan=scene_19_plan,
    scene_number=19
)

print(scene_19_script)

### 🗂️ Adding Metadata to Vector Database


In [ ]:
# Store the generated Scene 19 script in the vector database for future retrieval and context
store_scene_in_vector_db(
    client=client,
    sitcom_title=sitcom_title,
    scene_script=scene_19_script,
    embedding_model=embedding_model,
    index=index,
    vector_metadata=vector_metadata
)

## 🎬 Scene 20 Generation


In [ ]:
# Extract the description for Scene 20 from the episode outline
scene_20_desc = extract_scene(episode_outline, 20)
print(scene_20_desc)

### 🎭 Character Agent


In [ ]:
# Re-initialize the character agent with the updated vector metadata
character_agent = CharacterAgent(
    client=client,
    vector_metadata=vector_metadata,
    num_scenes=3
)

# Evaluate Scene 20 using the character agent and store the evaluation results
char_histories, char_is_consistent, char_explanation, char_recommendations, char_thoughts = evaluate_character_agent_scene(
    agent=character_agent,
    scene_description=scene_20_desc,
    scene_number=20
)

### 🎤 Comedic Agent


In [ ]:
# Re-initialize the comedy agent with the updated vector metadata
comedic_agent = ComedicAgent(
    client=client,
    vector_metadata=vector_metadata,
    num_scenes=3
)

# Evaluate Scene 20 using the comedy agent and store the evaluation results
com_context, com_is_consistent, com_analysis_text, com_recommendations, com_thoughts = evaluate_comedic_agent_scene(
    agent=comedic_agent,
    scene_description=scene_20_desc,
    scene_number=20
)

### 🌆 Environment Agent


In [ ]:
# Re-initialize the environment agent with the updated vector metadata
environment_agent = EnvironmentAgent(
    client=client,
    vector_metadata=vector_metadata,
    num_scenes=3
)

# Evaluate Scene 20 using the environment agent and store the evaluation results
context, is_consistent, explanation, env_recommendations, env_thoughts = evaluate_environment_agent_scene(
    agent=environment_agent,
    scene_description=scene_20_desc,
    scene_number=20
)

### 🧠 Scene Planner Agent


In [ ]:
# Generate a plan for Scene 20 by integrating recommendations from all three agents
scene_20_plan = scene_planner_agent.plan_next_scene(
    character_recommendations=char_recommendations,
    comedic_recommendations=com_recommendations,
    environment_recommendations=env_recommendations,
    scene_number=20
)

print(scene_20_plan)

### ✍️ Generating Scene 20 Script

In [ ]:
# Generate the full script for Scene 20 based on the structured scene plan
scene_20_script = generate_scene(
    client=client,
    scene_plan=scene_20_plan,
    scene_number=20
)

print(scene_20_script)

## Combining Scripts



In [ ]:
# List your scene scripts in order
scene_scripts = [
    scene_1_script,
    scene_2_script,
    scene_3_script,
    scene_4_script,
    scene_5_script,
    scene_6_script,
    scene_7_script,
    scene_8_script,
    scene_9_script,
    scene_10_script,
    scene_11_script,
    scene_12_script,
    scene_13_script,
    scene_14_script,
    scene_15_script,
    scene_16_script,
    scene_17_script,
    scene_18_script,
    scene_19_script,
    scene_20_script
]

# Combine with optional scene headers
full_episode_script = "\n\n".join([
    f"### Scene {i+1} ###\n{script.strip()}"
    for i, script in enumerate(scene_scripts)
])

# Preview the first 500 characters
print(full_episode_script[:500])


### Scene 1 ###
INT. ROBBIE'S APARTMENT ABOVE THE LOCKSMITH SHOP - MORNING 

[Camera pans across a chaotic room filled with lock-picking tools, locksmith manuals, and an unmade bed. An ALARM CLOCK is ringing loudly.]

[Focusing on the bed, a lump of blankets stirs. Suddenly, a hand shoots out, groping for the alarm clock.]

ROBBIE 
(muffled, from under the covers)
Alright, alright, I hear ya!

[ROBBIE, mid-thirties, disheveled, pops his head out from under the covers. He squints at the alarm clock.]

ROBBIE
(grumbling to himself)
Six o'clock... Why did I set it for six o'clock?

[LAUGH TRACK]

[ROBBIE fumbles for his glasses on the bedside table, knocking over a pile of locksmith manuals in the process.]

ROBBIE
(startled)
Whoa!

[LAUGH TRACK]

[ROBBIE finds his glasses, puts them on, then peers at the alarm clock again.]

ROBBIE
(squinting)
Oh, eight o'clock. That's more like it.

[LAUGH TRACK]

[Camera pans to a calendar on the wall. The date circled is today, and it's clearly marked

In [ ]:
stop

In [ ]:
path = "/content/drive/MyDrive/Spring 2025/Gen AI with LLM/Project/episode data/full_episode_script.txt"

with open(path, "w") as f:
    f.write(full_episode_script)

print(f"Script saved to {path}")

Script saved to /content/drive/MyDrive/Spring 2025/Gen AI with LLM/Project/episode data/full_episode_script.txt
